In [1]:
# import libraries
from pyopenms import * # main package used for handling MS data
import os # changing directories
import pandas as pd # creating and manipulating dataframe

 # importing variables and functions from other scripts
from constants import ION_TYPES, DEFAULT_MAX_CHARGE
import match as m
import tensorize as tens
from tensorize import csv, csv_training
from io_local import to_hdf5, from_hdf5

In [2]:
def formatData(mzML, log, realtime):
    global test
    # load the content of the mzML file into the exp variable of type MSExperiment
    exp = MSExperiment()
    MzMLFile().load(mzML, exp)
    
    # loop through the spectra to filter MS3 scans
    specM3 = [] # list of MS3 spectra 
    row_data = []
    for s in exp.getSpectra():
        if s.getMSLevel() == 3:
            specM3.append(s)
            # get scan number
            s_number = s.getNativeID().split(' ')[-1]
            _, scan_number = s_number.split('=')
            # obtain mz and intensity values 
            mz, intensity = s.get_peaks()
            mz_mod = " ".join(str(m) for m in mz)
            intensity_mod = " ".join(str(i) for i in intensity)
            # create dict (rows of dataframe)
            data = {#'MS3_Scan':scan_number,
           'masses_raw':mz_mod,
           'intensities_raw':intensity_mod}
            row_data.append(data)
    print("num MS3s: ", len(specM3))
            
    # create series of all MS3 spectra
    s_series = pd.Series(specM3)
    # create dataframe
    df = pd.DataFrame(row_data)
    
    # parse functions
    def parseScanLine(input):
        x = input.split(" For: ")
        [scan_number, mzs] = x[1].split(", ")
        [precursor_mz, fragment_mz] = mzs.split(";")
        trimmed_fragment_mz = fragment_mz.strip() # trim fragment strings to remove \n
        return [scan_number, precursor_mz, trimmed_fragment_mz]

    def parseTargetIons(input):
        i = input.split('Target Fragment: ')
        ion = i[1].split(',')[0]
        return ion 
    
    # checking lines of log file and creating dictionary of scan numbers and fragment mzs
    try:
        # words to search for
        search = ' Submitted Custom Scan For:'
        search_target = 'Target Fragment:'
        # dict for scan numbers and corresponding fragments 
        scan2frag = dict()
        target_values = []
        with open(log) as f:
            for line in f:
                if search in line:
                    scan_number, precursor_mz, trimmed_fragment_mz = parseScanLine(line)
                    scan2frag[scan_number] = [float(precursor_mz), float(trimmed_fragment_mz)]
                elif search_target in line:
                    target_ion = parseTargetIons(line)
                    target_values.append(target_ion) #to add to final dataframe
        # if the input string doesn't exist in the text file
        if len(scan2frag)==0:
            print("\n\"" + search + "\" is not found in \"" + log + "\"!")
    except FileNotFoundError:
        print("The file does not exist!")
        
    # obtain MS3 scan numbers
    # obtain precursor and fragment mzs directly from the MS3 spectrum
    ms3scan2MZs = dict()
    for s in specM3:
        s_number = s.getNativeID().split(' ')[-1]
        _, scan_number = s_number.split('=')
        fragment, precursor = s.getPrecursors()
        precursor_mz = precursor.getMZ()
        fragment_mz = fragment.getMZ()
        ms3scan2MZs[int(scan_number)] = [round(float(precursor_mz), 4), round(float(fragment_mz), 4)] # 4 decimal places, similar to log
        
    def matchingMS3s(ms2_mzs, ms3_mzs): # either fragment or precursor
        # making sure they are within 100 scans of each other
        too_far = []
        for ms2scan, ms3scan in zip(list(scan2frag), list(ms3scan2MZs)):
            scan_diff = int(ms3scan) - int(ms2scan)
            if scan_diff > 100:
                too_far.append('Scans are not within 100 scans of each other...' + 'MS2 = ' + str(ms2scan) + ' MS3 = ' + str(ms3scan))
        # do they not match off the bat?
        if ms2_mzs != ms3_mzs:
            # taking into consideration rounding discrepencies between the log and the spectrum
            mismatch = []
            for i in range(0, len(list(ms3scan2MZs))):
                precursor_diff = float(list(ms3scan2MZs.values())[i][0]) - float(list(scan2frag.values())[i][0]) 
                if precursor_diff < 0.000101 or (precursor_diff < 0 and precursor_diff > -0.000101): # because sometimes max number will be 0.0001000002 for example
                    pass
                else:
                    mismatch.append(i)
                fragment_diff = float(list(ms3scan2MZs.values())[i][1]) - float(list(scan2frag.values())[i][1]) 
                if fragment_diff < 0.000101 or (fragment_diff < 0 and fragment_diff > -0.000101): # because sometimes max number will be 0.0001000002 for example
                    pass
                else:
                    mismatch.append(i)
            # no mismatch after rounding and within 100 scans
            if len(mismatch) == len(too_far) == 0:
                print('Scans match up after taking rounding discrepencies into consideration')
                ms2_scans = list(scan2frag)
                return ms2_scans
            elif len(mismatch) != 0:
                print('There is mismatch at the following indicies:') # if this is the case, need to do more work...
                for i in mismatch:
                    print(i) 
            elif len(too_far) != 0:
                print(too_far)
        # they match perfectly
        elif ms2_mzs == ms3_mzs:
            # within 100 scans
            if len(too_far) == 0:
                print('Scans match up perfectly!')
                ms2_scans = list(scan2frag)
                return ms2_scans
            else:
                print(too_far)
    
    # make sure that MS3 scans are in the same order as MS2 scans
    ms2_scans = matchingMS3s(list(ms3scan2MZs.values()), list(scan2frag.values()))
    
    # use realtime file to obtain peptide sequence and charge
    # read in peptide sequence from tsv
    tsv = pd.read_csv(realtime, sep='\t')
    # create dictionary with scan # as key and sequence/charge as values
    scan2PeptideCharge = dict([(i, [x,y]) for i, x,y, in zip(tsv['Scan Number'], tsv['Peptide'], tsv['Charge State'])])
    # removing all NaN sequences (not useful)
    scan2PeptideCharge_modified = {k:v for k,v in scan2PeptideCharge.items() if str(v[0]) != 'nan'}
    
    # collect data for dataframe
    seqs = []
    charges = []
    analyzer = []
    collision = []
    energy = int(realtime.split('_')[-2][-2:])
    for scan in ms2_scans:
        if int(scan) in list(scan2PeptideCharge_modified):
            charge = scan2PeptideCharge_modified[int(scan)][1]
            charges.append(charge)
            sequence = scan2PeptideCharge_modified[int(scan)][0]     
            #trimmed_sequence = sequence[2:-2] # remove first two and last two characters 
            seqs.append(sequence)
            # all ms3 scans are orbit trap 
            # to be added to dataframe with other MS3 info
            analyzer.append('FTMS')
            # all scans have same collision energy
            collision.append(energy)
    
    # add all data columns to dataframe
    df = df.assign(charge=charges, modified_sequence=seqs, mass_analyzer=analyzer, collision_energy=collision, target_fragment=target_values)
    
    # remove all modified sequences
    for i in df.index:
        if ('[' or ']') in df['modified_sequence'][i]:
            df.drop(i, axis=0, inplace=True)
            s_series.drop(i, inplace=True)
            
    # obtain target fragment sequences
    fragment_seqs = []
    for i in df.index:
        trimmed_seq = df['modified_sequence'][i][2:-2]
        peptide_object = AASequence.fromString(trimmed_seq) 
        # targeted fragment is a y ion
        if df['target_fragment'][i].startswith('y'):
            y_num = df['target_fragment'][i][-1]
            # the full sequence of the fragment
            full_seq = peptide_object.getSuffix(int(y_num))
        # targeted fragment is a b ion
        elif df['target_fragment'][i].startswith('b'):
            b_num = df['target_fragment'][i][-1]
            # the full sequence of the fragment
            full_seq = peptide_object.getPrefix(int(b_num))
        fragment_seqs.append(str(full_seq))
    df = df.assign(modified_sequence=fragment_seqs)
    
    # remove all sequences more than 30 in length
    for i in df.index:
        if len(df['modified_sequence'][i]) >= 30:
            df.drop(i, axis=0, inplace=True)
            s_series.drop(i, inplace=True)
            
    test = df.copy()
    
    # running augment function
    df_augmented = m.augment(df, ION_TYPES, DEFAULT_MAX_CHARGE)
    # remove all rows that are completely empty
    for i in df_augmented.index:
        if df_augmented['matches_charge1'][i] == df_augmented['matches_charge2'][i] == df_augmented['matches_charge3'][i] == df_augmented['matches_charge4'][i] == df_augmented['matches_charge5'][i] == df_augmented['matches_charge6'][i]:
            df_augmented.drop(i, axis=0, inplace=True)
            s_series.drop(i, inplace=True)
    
    return df_augmented, s_series

In [3]:
# change directory to find data files of interest
os.chdir(r'C:\Users\miar\Desktop\data')

In [4]:
# experiment files 
mzML = 'HEK293T_De_Novo_061122_Glu-C_B_BP_anyLength_HCD10.mzML'
log = 'App-2022-06-12_14-16-26.log'
realtime = 'HEK293T_De_Novo_061122_Glu-C_B_BP_anyLength_HCD10_realtimesearch.tsv'

In [5]:
df1, series1 = formatData(mzML, log, realtime)

num MS3s:  944
Scans match up after taking rounding discrepencies into consideration
b1 58.028740467
b2 221.09206946700002
b3 334.176133467
y1 148.060434167
y2 261.144498167
y3 424.207827167
b1 29.518008467
b2 111.049672967
b3 167.591704967
y1 74.533855317
y2 131.075887317
y3 212.607551817
b1 20.014431133666665
b2 74.36887413366667
b3 112.06356213366666
y1 50.024995700333335
y2 87.71968370033333
y3 142.07412670033332
b1 15.262642467000001
b2 56.028474717
b3 84.299490717
y1 37.770565892
y2 66.041581892
y3 106.807414142
b1 129.102239467
b2 285.203350467
y1 98.060041167
y2 254.161152167
b1 65.054757967
b2 143.105313467
y1 49.533658816999996
y2 127.584214317
b1 43.705597467
b2 95.73930113366667
y1 33.35819803366667
y2 85.39190170033334
b1 33.031017217
b2 72.056294967
y1 25.270467642
y2 64.295745392
b1 114.091340467
b2 270.192451467
y1 148.060434167
y2 304.161545167
b1 57.549308466999996
b2 135.599863967
y1 74.533855317
y2 152.584410817
b1 38.701964467
b2 90.73566813366666
y1 50.02499570033

b4 250.1384963288
y1 44.523290816999996
y2 101.06532281699998
y3 149.591704817
y4 229.6070291788
b1 98.060040467
b2 212.10296746699998
b3 368.204078467
b4 496.299041467
y1 148.060434167
y2 276.155397167
y3 432.256508167
y4 546.2994351670001
b1 49.533658466999995
b2 106.55512196699999
b3 184.605677467
b4 248.653158967
y1 74.533855317
y2 138.581336817
y3 216.631892317
y4 273.65335581700003
b1 33.358197800333336
b2 71.37250680033333
b3 123.406210467
b4 166.104531467
y1 50.024995700333335
y2 92.72331670033333
y3 144.757020367
y4 182.77132936700002
b1 115.050203467
y1 114.091341167
b1 58.028739967
y1 57.549308817
b1 102.054955467
y1 148.060434167
b1 51.531115967
y1 74.533855317
b1 34.68983613366667
y1 50.024995700333335
b1 26.269196217
y1 37.770565892
b1 88.039304467
y1 115.050204167
b1 44.523290466999995
y1 58.028740317
b1 100.075690467
b2 256.176801467
b3 355.24521546700004
b4 515.2758641906
y1 148.060434167
y2 308.0910828906
y3 407.1594968906
y4 563.2606078906
b1 50.541483467
b2 128.5920

b1 157.108387467
b2 313.209498467
y1 72.044391167
y2 228.14550216700002
b1 79.057831967
b2 157.108387467
y1 36.525833817
y2 114.57638931700001
b1 53.040980133666665
b2 105.07468380033333
y1 24.686314700333337
y2 76.72001836700001
b1 40.032554217
b2 79.057831967
y1 18.766555142
y2 57.791832892
b1 114.091340467
b2 227.175404467
b3 355.270367467
y1 102.054956167
y2 230.149919167
y3 343.233983167
b1 57.549308466999996
b2 114.091340467
b3 178.138821967
y1 51.531116317
y2 115.578597817
y3 172.120629817
b1 129.102239467
b2 186.123703467
b3 287.171382467
y1 187.086590167
y2 288.134269167
y3 345.155733167
b1 65.054757967
b2 93.565489967
b3 144.089329467
y1 94.046933317
y2 144.570772817
y3 173.081504817
b1 43.705597467
b2 62.71275213366667
b3 96.39531180033333
y1 63.033714366999995
y2 96.71627403366666
y3 115.72342870033333
b1 33.031017217
b2 47.286383217
b3 72.548302967
y1 47.527104892
y2 72.789024642
y3 87.044390642
b1 102.054955467
b2 173.092069467
b3 301.150647467
b4 388.18267546699997
y1 11

b2 68.032261467
b3 101.71482113366666
b4 150.73762580033335
y1 53.040980366999996
y2 102.06378503366666
y3 135.74634470033334
y4 164.75702070033333
b1 138.066188467
b2 294.167299467
y1 134.044784167
y2 290.145895167
b1 69.536732467
b2 147.587287967
y1 67.526030317
y2 145.576585817
b1 46.693580467
b2 98.72728413366667
y1 45.353112366999994
y2 97.38681603366666
b1 35.272004466999995
b2 74.297282217
y1 34.266653391999995
y2 73.291931142
b1 28.419058867
b2 59.639281067
y1 27.614778007
y2 58.83500020699999
b1 138.066188467
b2 237.13460246699998
b3 338.182281467
b4 437.250695467
b5 536.319109467
y1 175.118952167
y2 274.187366167
y3 373.255780167
y4 474.303459167
y5 573.3718731670001
b1 69.536732467
b2 119.07093946699999
b3 169.594778967
b4 219.128985967
b5 268.663192967
y1 88.063114317
y2 137.597321317
y3 187.131528317
y4 237.655367817
y5 287.18957481700005
b1 46.693580467
b2 79.71638513366666
b3 113.39894480033333
b4 146.421749467
b5 179.44455413366666
y1 59.044501700333335
y2 92.067306367


b1 129.102239467
b2 257.197202467
b3 404.265616467
y1 148.060434167
y2 295.12884816699994
y3 423.22381116699995
b1 65.054757967
b2 129.102239467
b3 202.63644646699998
y1 74.533855317
y2 148.06806231699997
y3 212.11554381699997
b1 58.028740467
b2 195.087652467
y1 129.065855167
y2 266.12476716699996
b1 29.518008467
b2 98.047464467
y1 65.036565817
y2 133.56602181699998
b1 115.050203467
b2 186.087317467
y1 100.075691167
y2 171.11280516699998
b1 58.028739967
b2 93.547296967
y1 50.541483817
y2 86.06004081699999
b1 114.091340467
b2 201.12336846699998
y1 58.028741167
y2 145.060769167
b1 57.549308466999996
b2 101.06532246699999
y1 29.518008817000002
y2 73.034022817
b1 38.701964467
b2 67.71264046699999
y1 20.014431367
y2 49.025107367
b1 114.091340467
b2 201.12336846699998
y1 88.039305167
y2 175.07133316699998
b1 57.549308466999996
b2 101.06532246699999
y1 44.523290816999996
y2 88.03930481699999
b1 38.701964467
b2 67.71264046699999
y1 30.017952700333336
y2 59.02862870033332
b1 114.091340467
b2 21

...

y2 215.139020167
y3 343.233983167
b1 58.028739967
b2 122.07622146700001
b3 172.60006096700002
y1 57.549308817
y2 108.073148317
y3 172.120629817
b1 39.021585467
b2 81.719906467
b3 115.40246613366668
y1 38.701964700333335
y2 72.384524367
y3 115.082845367
b1 29.518008217000002
b2 61.541748967000004
b3 86.80366871700001
y1 29.278292642
y2 54.540212392
y3 86.563953142
b1 23.815861867000002
b2 49.43485446700001
b3 69.644390267
y1 23.624089407
y2 43.833625207000004
y3 69.452617807
b1 88.039304467
y1 138.066189167
b1 44.523290466999995
y1 69.536732817
b1 30.017952467
y1 46.693580700333335
b1 22.765283467
y1 35.272004642
b1 18.413682067
y1 28.419059007000005
b1 88.039304467
b2 244.140415467
y1 138.066189167
y2 294.167300167
b1 44.523290466999995
b2 122.573845967
y1 69.536732817
y2 147.587288317
b1 30.017952467
b2 82.05165613366667
y1 46.693580700333335
y2 98.727284367
b1 22.765283467
b2 61.790561217
y1 35.272004642
y2 74.297282392
b1 72.044390467
b2 219.112804467
y1 148.060434167
y2 295.1288481

b1 29.518008217000002
y1 37.774380142
b1 138.066188467
b2 266.124766467
b3 379.208830467
y1 161.03792589059998
y2 274.1219898906
y3 402.1805678906
b1 69.536732467
b2 133.56602146699998
b3 190.108053467
y1 81.02260117879999
y2 137.5646331788
y3 201.5939221788
b1 46.693580467
b2 89.37977313366666
b3 127.07446113366666
y1 54.350826274866655
y2 92.04551427486666
y3 134.73170694153333
b1 88.039304467
y1 157.108388167
b1 44.523290466999995
y1 79.05783231699999
b1 30.017952467
y1 53.040980366999996
b1 22.765283467
y1 40.032554391999994
b1 72.044390467
y1 182.081170167
b1 36.525833467
y1 91.544223317
b1 24.686314467000003
y1 61.36524103366667
b1 18.766554967
y1 46.275749892
b1 15.214699267
y1 37.222055207000004
b1 12.846795467
y1 31.186258750333337
b1 114.091340467
b2 227.175404467
b3 340.259468467
y1 132.04776216699997
y2 245.13182616699999
y3 358.215890167
b1 57.549308466999996
b2 114.091340467
b3 170.633372467
y1 66.52751931699999
y2 123.06955131699999
y3 179.611583317
b1 38.701964467
b2 76

b1 148.075690467
b2 261.15975446699997
b3 374.243818467
b4 431.26528246699996
b5 518.297310467
y1 148.060434167
y2 235.092462167
y3 292.113926167
y4 405.197990167
y5 518.2820541670001
b1 74.54148346699999
b2 131.08351546699998
b3 187.625547467
b4 216.13627946699998
b5 259.652293467
y1 74.533855317
y2 118.049869317
y3 146.560601317
y4 203.102633317
y5 259.64466531700003
b1 50.03008113366666
b2 87.72476913366665
b3 125.41945713366665
b4 144.42661180033332
b5 173.43728780033334
y1 50.024995700333335
y2 79.03567170033334
y3 98.04282636699999
y4 135.737514367
y5 173.43220236700003
b1 37.774379966999994
b2 66.04539596699999
b3 94.316411967
b4 108.57177796699999
b5 130.329784967
y1 37.770565892
y2 59.528572892
y3 73.783938892
y4 102.054954892
y5 130.32597089200002
b1 98.060040467
y1 148.060434167
b1 49.533658466999995
y1 74.533855317
b1 33.358197800333336
y1 50.024995700333335
b1 102.054955467
b2 159.076419467
b3 272.160483467
b4 343.19759746700004
y1 129.065855167
y2 200.10296916700003
y3 31

b1 132.047761467
b2 269.10667346699995
b3 425.20778446699995
y1 148.060434167
y2 304.161545167
y3 441.220457167
b1 66.52751896699999
b2 135.05697496699997
b3 213.10753046699998
y1 74.533855317
y2 152.584410817
y3 221.113866817
b1 44.68743813366666
b2 90.37374213366665
b3 142.4074458003333
y1 50.024995700333335
y2 102.05869936699999
y3 147.745003367
b1 33.767397716999994
b2 68.03212571699999
b3 107.05740346699999
y1 37.770565892
y2 76.795843642
y3 111.060571642
b1 100.075690467
b2 247.14410446699998
b3 375.202682467
b4 488.286746467
b5 616.3453244670001
y1 72.044391167
y2 200.10296916700003
y3 313.187033167
y4 441.245611167
y5 588.314025167
b1 50.541483467
b2 124.07569046699999
b3 188.104979467
b4 244.647011467
b5 308.67630046700003
y1 36.525833817
y2 100.55512281700001
y3 157.097154817
y4 221.126443817
y5 294.660650817
b1 34.03008113366667
b2 83.05288580033333
b3 125.739078467
b4 163.433766467
b5 206.1199591336667
y1 24.686314700333337
y2 67.37250736700001
y3 105.067195367
y4 147.75338

b1 22.765283467
b2 47.028474466999995
y1 33.021921142
y2 57.285112142
b1 114.091340467
b2 270.192451467
y1 134.044784167
y2 290.145895167
b1 57.549308466999996
b2 135.599863967
y1 67.526030317
y2 145.576585817
b1 38.701964467
b2 90.73566813366666
y1 45.353112366999994
y2 97.38681603366666
b1 29.278292467
b2 68.303570217
y1 34.266653391999995
y2 73.291931142
b1 114.091340467
b2 228.134267467
y1 129.102240167
y2 243.145167167
b1 57.549308466999996
b2 114.570771967
y1 65.054758317
y2 122.076221817
b1 38.701964467
b2 76.716273467
y1 43.70559770033333
y2 81.71990670033334
b1 29.278292467
b2 57.789024217
y1 33.031017391999995
y2 61.541749142
b1 157.108387467
b2 258.156066467
y1 100.075691167
y2 201.12337016700002
b1 79.057831967
b2 129.581671467
y1 50.541483817
y2 101.06532331700001
b1 53.040980133666665
b2 86.72353980033334
y1 34.030081367
y2 67.71264103366667
b1 40.032554217
b2 65.294473967
y1 25.774380142000002
y2 51.036299892
b1 157.108387467
y1 114.091341167
b1 79.057831967
y1 57.549308

In [6]:
print(len(test['masses_raw']))

840


In [7]:
# experiment files 
mzML = 'HEK293T_De_Novo_061122_Glu-C_B_BP_anyLength_HCD15.mzML'
log = 'App-2022-06-12_09-54-27.log'
realtime = 'HEK293T_De_Novo_061122_Glu-C_B_BP_anyLength_HCD15_realtimesearch.tsv'

In [8]:
df2, series2 = formatData(mzML, log, realtime)

num MS3s:  960
Scans match up after taking rounding discrepencies into consideration
b1 102.054955467
b2 189.08698346699998
b3 302.171047467
y1 148.060434167
y2 261.144498167
y3 348.17652616699996
b1 51.531115967
b2 95.04712996699999
b3 151.589161967
y1 74.533855317
y2 131.075887317
y3 174.59190131699998
b1 34.68983613366667
b2 63.700512133666656
b3 101.39520013366666
y1 50.024995700333335
y2 87.71968370033333
y3 116.73035970033332
b1 26.269196217
b2 48.02720321699999
b3 76.298219217
y1 37.770565892
y2 66.041581892
y3 87.79958889199999
b1 164.070605467
b2 221.09206946700002
y1 148.060434167
y2 205.08189816700002
b1 82.538940967
b2 111.049672967
y1 74.533855317
y2 103.04458731700001
b1 55.361719467
b2 74.36887413366667
y1 50.024995700333335
y2 69.032150367
b1 41.773108717
b2 56.028474717
y1 37.770565892
y2 52.025931892
b1 114.091340467
b2 270.192451467
y1 134.044784167
y2 290.145895167
b1 57.549308466999996
b2 135.599863967
y1 67.526030317
y2 145.576585817
b1 38.701964467
b2 90.73566813

y2 205.08189816700002
b1 49.533658466999995
b2 78.044390467
y1 74.533855317
y2 103.04458731700001
b1 98.060040467
b2 155.081504467
y1 148.060434167
y2 205.08189816700002
b1 49.533658466999995
b2 78.044390467
y1 74.533855317
y2 103.04458731700001
b1 100.075690467
b2 213.159754467
b3 341.218332467
b4 440.286746467
y1 148.060434167
y2 247.12884816700003
y3 375.187426167
y4 488.271490167
b1 50.541483467
b2 107.083515467
b3 171.112804467
b4 220.647011467
y1 74.533855317
y2 124.06806231700001
y3 188.097351317
y4 244.639383317
b1 34.03008113366667
b2 71.72476913366667
b3 114.41096180033333
b4 147.433766467
y1 50.024995700333335
y2 83.04780036700001
y3 125.73399303366666
y4 163.42868103366666
b1 25.774379967
b2 54.045395967
b3 86.060040467
b4 110.827143967
y1 37.770565892
y2 62.537669392000005
y3 94.552313892
y4 122.823329892
b1 102.054955467
b2 189.08698346699998
b3 302.171047467
b4 458.272158467
y1 148.060434167
y2 304.161545167
y3 417.245609167
y4 504.277637167
b1 51.531115967
b2 95.0471299

b1 102.054955467
b2 173.092069467
b3 329.193180467
y1 148.060434167
y2 304.161545167
y3 375.198659167
b1 51.531115967
b2 87.04967296699999
b3 165.100228467
y1 74.533855317
y2 152.584410817
y3 188.102967817
b1 34.68983613366667
b2 58.36887413366666
b3 110.40257780033333
y1 50.024995700333335
y2 102.05869936699999
y3 125.73773736700001
b1 26.269196217
b2 44.028474716999995
b3 83.053752467
y1 37.770565892
y2 76.795843642
y3 94.555122142
b1 21.216812267
b2 35.424235067
b3 66.64445726699999
y1 30.417908007000005
y2 61.638130206999996
y3 75.845553007
b1 98.060040467
b2 195.112804467
b3 351.213915467
y1 148.060434167
y2 304.161545167
y3 401.21430916699995
b1 49.533658466999995
b2 98.060040467
b3 176.110595967
y1 74.533855317
y2 152.584410817
y3 201.11079281699998
b1 33.358197800333336
b2 65.70911913366666
b3 117.74282280033333
y1 50.024995700333335
y2 102.05869936699999
y3 134.4096207003333
b1 25.270467467
b2 49.533658466999995
b3 88.558936217
y1 37.770565892
y2 76.795843642
y3 101.0590346419

b1 58.028739967
b2 107.562946967
b3 164.104978967
y1 44.523290816999996
y2 101.06532281699998
y3 150.599529817
b1 39.021585467
b2 72.04439013366667
b3 109.73907813366667
y1 30.017952700333336
y2 67.71264070033332
y3 100.735445367
b1 161.0379251906
b2 274.1219891906
b3 361.15401719060003
b4 474.23808119060004
y1 98.060041167
y2 211.14410516699996
y3 298.176133167
y4 411.260197167
b1 81.0226008288
b2 137.5646328288
b3 181.0806468288
b4 237.62267882880002
y1 49.533658816999996
y2 106.07569081699998
y3 149.591704817
y4 206.133736817
b1 54.35082604153333
b2 92.04551404153334
b3 121.05619004153334
b4 158.75087804153335
y1 33.35819803366667
y2 71.05288603366665
y3 100.06356203366666
y4 137.75825003366666
b1 98.060040467
b2 199.10771946699998
b3 256.129183467
y1 148.060434167
y2 205.08189816700002
y3 306.129577167
b1 49.533658466999995
b2 100.05749796699999
b3 128.568229967
y1 74.533855317
y2 103.04458731700001
y3 153.568426817
b1 98.060040467
b2 254.161151467
y1 148.060434167
y2 304.161545167

y1 69.536732817
b1 63.03371413366667
y1 46.693580700333335
b1 47.527104717
y1 35.272004642
b1 58.028740467
b2 129.065854467
y1 132.04776216699997
y2 203.084876167
b1 29.518008467
b2 65.036565467
y1 66.52751931699999
y2 102.046076317
b1 20.014431133666665
b2 43.69346913366667
y1 44.68743836699999
y2 68.366476367
b1 88.039304467
y1 148.060434167
b1 44.523290466999995
y1 74.533855317
b1 114.091340467
b2 227.175404467
b3 383.27651546699997
b4 440.29797946699995
y1 138.066189167
y2 195.08765316699998
y3 351.188764167
y4 464.272828167
b1 57.549308466999996
b2 114.091340467
b3 192.14189596699998
b4 220.65262796699997
y1 69.536732817
y2 98.04746481699999
y3 176.098020317
y4 232.640052317
b1 38.701964467
b2 76.396652467
b3 128.43035613366666
b4 147.4375108003333
y1 46.693580700333335
y2 65.700735367
y3 117.73443903366666
y4 155.42912703366667
b1 129.102239467
b2 200.139353467
y1 114.091341167
y2 185.12845516699997
b1 65.054757967
b2 100.573314967
y1 57.549308817
y2 93.06786581699998
b1 43.70559

...

b5 589.2769094669999
y1 148.060434167
y2 334.139747167
y3 405.176861167
y4 492.208889167
y5 639.277303167
b1 49.533658466999995
b2 123.06786546699999
b3 166.58387946699997
b4 202.10243646699996
b5 295.14209296699994
y1 74.533855317
y2 167.573511817
y3 203.092068817
y4 246.608082817
y5 320.142289817
b1 33.358197800333336
b2 82.38100246699999
b3 111.39167846699998
b4 135.07071646699998
b5 197.09715413366663
y1 50.024995700333335
y2 112.051433367
y3 135.730471367
y4 164.741147367
y5 213.76395203366667
b1 25.270467467
b2 62.03757096699999
b3 83.79557796699999
b4 101.55485646699998
b5 148.07468471699997
y1 37.770565892
y2 84.290394142
y3 102.049672642
y4 123.807679642
y5 160.574783142
b1 129.102239467
b2 226.155003467
b3 283.176467467
y1 98.060041167
y2 155.081505167
y3 252.134269167
b1 65.054757967
b2 113.581139967
b3 142.091871967
y1 49.533658816999996
y2 78.044390817
y3 126.570772817
b1 43.705597467
b2 76.05651880033334
b3 95.063673467
y1 33.35819803366667
y2 52.365352700333325
y3 84.716

b1 100.075690467
b2 237.13460246699998
b3 350.218666467
y1 148.060434167
y2 261.144498167
y3 398.20341016699996
b1 50.541483467
b2 119.07093946699999
b3 175.612971467
y1 74.533855317
y2 131.075887317
y3 199.60534331699998
b1 34.03008113366667
b2 79.71638513366666
b3 117.41107313366666
y1 50.024995700333335
y2 87.71968370033333
y3 133.40598770033333
b1 102.054955467
b2 230.14991846700002
b3 358.244881467
b4 486.33984446700003
y1 148.075691167
y2 276.17065416699995
y3 404.26561716699996
y4 532.360580167
b1 51.531115967
b2 115.57859746700001
b3 179.626078967
b4 243.67356046700002
y1 74.541483817
y2 138.58896531699997
y3 202.63644681699998
y4 266.683928317
b1 34.68983613366667
b2 77.38815713366667
b3 120.08647813366667
b4 162.78479913366667
y1 50.030081367
y2 92.72840236699999
y3 135.426723367
y4 178.125044367
b1 26.269196217
b2 58.292936967
b3 90.316677717
b4 122.340418467
y1 37.774380142
y2 69.79812089199999
y3 101.82186164199999
y4 133.845602392
b1 98.060040467
b2 226.155003467
b3 354.2

y1 34.266653391999995
b1 115.050203467
b2 228.134267467
b3 388.1649161906
y1 58.028741167
y2 218.05938989060002
y3 331.1434538906
b1 58.028739967
b2 114.570771967
b3 194.5860963288
y1 29.518008817000002
y2 109.5333331788
y3 166.07536517879998
b1 39.021585467
b2 76.716273467
b3 130.05982304153335
y1 20.014431367
y2 73.35798094153334
y3 111.05266894153333
b1 132.047761467
b2 189.069225467
b3 286.121989467
b4 373.154017467
b5 444.191131467
y1 148.060434167
y2 219.097548167
y3 306.129576167
y4 403.18234016699995
y5 460.20380416699993
b1 66.52751896699999
b2 95.038250967
b3 143.564632967
b4 187.080646967
b5 222.599203967
y1 74.533855317
y2 110.052412317
y3 153.568426317
y4 202.09480831699997
y5 230.60554031699996
b1 44.68743813366666
b2 63.69459280033333
b3 96.04551413366666
b4 125.05619013366668
b5 148.73522813366665
y1 50.024995700333335
y2 73.70403370033334
y3 102.71470970033333
y4 135.06563103366665
y5 154.0727857003333
b1 132.047761467
b2 203.084875467
b3 331.143453467
b4 487.244564467

y3 327.202682167
y4 455.297645167
b1 50.541483467
b2 114.58896496700001
b3 171.13099696700002
b4 220.66520396700002
y1 58.028740317
y2 107.56294731700001
y3 164.104979317
y4 228.152460817
b1 34.03008113366667
b2 76.72840213366668
b3 114.42309013366668
b4 147.44589480033335
y1 39.02158570033334
y2 72.044390367
y3 109.73907836699999
y4 152.43739936699998
b1 25.774379967
b2 57.798120717
b3 86.069136717
b4 110.83624021700001
y1 29.518008392000002
y2 54.285111892
y3 82.55612789199999
y4 114.579868642
b1 98.060040467
b2 235.11895246699999
y1 148.060434167
y2 285.11934616699995
b1 49.533658466999995
b2 118.06311446699999
y1 74.533855317
y2 143.06331131699997
b1 33.358197800333336
b2 79.04450180033332
y1 50.024995700333335
y2 95.71129970033331
b1 25.270467467
b2 59.535195466999994
y1 37.770565892
y2 72.03529389199998
b1 114.091340467
b2 201.12336846699998
y1 115.050204167
y2 202.08223216699997
b1 57.549308466999996
b2 101.06532246699999
y1 58.028740317
y2 101.54475431699998
b1 38.701964467
b2 

y3 101.3830717003333


In [9]:
print(len(test['masses_raw']))

809


In [10]:
# experiment files 
mzML = 'HEK293T_De_Novo_061122_Glu-C_B_BP_anyLength_HCD20.mzML'
log = 'App-2022-06-12_07-05-17.log'
realtime = 'HEK293T_De_Novo_061122_Glu-C_B_BP_anyLength_HCD20_realtimesearch.tsv'

In [11]:
df3, series3 = formatData(mzML, log, realtime)

num MS3s:  827
Scans match up after taking rounding discrepencies into consideration
b1 98.060040467
b2 169.097154467
b3 256.129182467
b4 343.161210467
y1 148.060434167
y2 235.092462167
y3 322.124490167
y4 393.161604167
b1 49.533658466999995
b2 85.052215467
b3 128.568229467
b4 172.084243467
y1 74.533855317
y2 118.049869317
y3 161.565883317
y4 197.084440317
b1 33.358197800333336
b2 57.03723580033333
b3 86.04791180033334
b4 115.05858780033333
y1 50.024995700333335
y2 79.03567170033334
y3 108.04634770033333
y4 131.72538570033333
b1 116.034219467
b2 213.08698346699998
b3 270.10844746699996
b4 357.140475467
y1 88.039305167
y2 175.07133316699998
y3 232.09279716700001
y4 329.145561167
b1 58.520747967
b2 107.04712996699999
b3 135.55786196699998
b4 179.073875967
y1 44.523290816999996
y2 88.03930481699999
y3 116.550036817
y4 165.076418817
b1 39.34959080033334
b2 71.70051213366666
b3 90.70766680033331
b4 119.71834280033333
y1 30.017952700333336
y2 59.02862870033332
y3 78.03578336700001
y4 110.386

b1 72.044390467
b2 228.145501467
b3 391.208830467
y1 98.060041167
y2 261.12337016699996
y3 417.22448116699996
b1 36.525833467
b2 114.576388967
b3 196.108053467
y1 49.533658816999996
y2 131.06532331699998
y3 209.11587881699998
b1 24.686314467000003
b2 76.72001813366667
b3 131.07446113366666
y1 33.35819803366667
y2 87.71264103366666
y3 139.7463447003333
b1 18.766554967
b2 57.791832717
b3 98.557664967
y1 25.270467642
y2 66.03629989199999
y3 105.06157764199999
b1 15.214699267
b2 46.434921467
b3 79.047587267
y1 20.417829407
y2 53.03049520699999
y3 84.250717407
b1 114.091340467
b2 171.112804467
b3 242.149918467
y1 115.050204167
y2 186.08731816699998
y3 243.10878216700002
b1 57.549308466999996
b2 86.060040467
b3 121.578597467
y1 58.028740317
y2 93.54729731699999
y3 122.058029317
b1 38.701964467
b2 57.70911913366666
b3 81.38815713366667
y1 39.02158570033334
y2 62.700623700333324
y3 81.707778367
b1 29.278292467
b2 43.533658466999995
b3 61.292936966999996
y1 29.518008392000002
y2 47.277286891999

b2 68.543285967
b3 93.805205717
y1 29.278292642
y2 54.540212392
y3 93.56549014199999
b1 100.075690467
y1 148.060434167
b1 50.541483467
y1 74.533855317
b1 88.039304467
y1 157.108388167
b1 44.523290466999995
y1 79.05783231699999
b1 115.050203467
y1 114.091341167
b1 58.028739967
y1 57.549308817
b1 39.021585467
y1 38.701964700333335
b1 29.518008217000002
y1 29.278292642
b1 72.044390467
b2 235.107719467
y1 132.04776216699997
y2 295.111091167
b1 36.525833467
b2 118.057497967
y1 66.52751931699999
y2 148.059183817
b1 24.686314467000003
b2 79.040757467
y1 44.68743836699999
y2 99.04188136699999
b1 18.766554967
b2 59.532387217
y1 33.76739789199999
y2 74.533230142
b1 15.214699267
b2 47.827365067
y1 27.215373606999997
y2 59.82803940699999
b1 129.065854467
y1 102.054956167
b1 65.036565467
y1 51.531116317
b1 43.69346913366667
y1 34.689836367000005
b1 33.021920967
y1 26.269196392
b1 26.618992067000004
y1 21.216812407
b1 58.028740467
b2 186.123703467
b3 273.155731467
y1 148.060434167
y2 235.092462167
y

y1 100.075691167
y2 197.12845516699997
y3 268.16556916699994
y4 424.26668016699995
b1 57.549308466999996
b2 135.599863967
b3 171.11842096700002
b4 219.64480296700003
y1 50.541483817
y2 99.06786581699998
y3 134.58642281699997
y4 212.63697831699997
b1 132.047761467
y1 164.070606167
b1 66.52751896699999
y1 82.538941317
b1 114.091340467
b2 270.192451467
b3 456.271764467
y1 148.075691167
y2 334.155004167
y3 490.256115167
b1 57.549308466999996
b2 135.599863967
b3 228.639520467
y1 74.541483817
y2 167.581140317
y3 245.631695817
b1 38.701964467
b2 90.73566813366666
b3 152.76210580033333
y1 50.030081367
y2 112.05651903366667
y3 164.09022270033333
b1 29.278292467
b2 68.303570217
b3 114.823398467
y1 37.774380142
y2 84.294208392
y3 123.319486142
b1 114.091340467
b2 227.175404467
b3 328.223083467
b4 475.291497467
b5 588.375561467
y1 129.102240167
y2 242.186304167
y3 389.25471816699996
y4 490.302397167
y5 603.386461167
b1 57.549308466999996
b2 114.091340467
b3 164.615179967
b4 238.149386967
b5 294.69

b2 202.08223146699999
b3 315.166295467
y1 134.044784167
y2 247.128848167
y3 361.17177516699996
b1 44.523290466999995
b2 101.54475396699999
b3 158.086785967
y1 67.526030317
y2 124.068062317
y3 181.08952581699998
b1 30.017952467
b2 68.032261467
b3 105.726949467
y1 45.353112366999994
y2 83.047800367
y3 121.06210936699999
b1 22.765283467
b2 51.276015216999994
b3 79.547031217
y1 34.266653391999995
y2 62.537669392
y3 91.04840114199999
b1 98.060040467
b2 226.118618467
b3 363.177530467
y1 148.060434167
y2 285.11934616699995
y3 413.17792416699996
b1 49.533658466999995
b2 113.562947467
b3 182.092403467
y1 74.533855317
y2 143.06331131699997
y3 207.09260031699998
b1 33.358197800333336
b2 76.044390467
b3 121.73069446699999
y1 50.024995700333335
y2 95.71129970033331
y3 138.39749236699998
b1 129.065854467
b2 257.124432467
b3 370.20849646700003
b4 441.245610467
y1 156.076753167
y2 227.113867167
y3 340.197931167
y4 468.256509167
b1 65.036565467
b2 129.065854467
b3 185.60788646700001
b4 221.126443467
y1

...

y1 33.76739789199999
y2 62.038413891999994
y3 90.309429892
b1 129.065854467
b2 243.10878146700003
b3 356.19284546700004
b4 503.26125946700006
y1 148.060434167
y2 295.12884816699994
y3 408.21291216699996
y4 522.255839167
b1 65.036565467
b2 122.05802896700001
b3 178.60006096700002
b4 252.13426796700003
y1 74.533855317
y2 148.06806231699997
y3 204.61009431699998
y4 261.631557817
b1 43.69346913366667
b2 81.70777813366668
b3 119.40246613366668
b4 168.42527080033335
y1 50.024995700333335
y2 99.04780036699998
y3 136.742488367
y4 174.756797367
b1 148.075690467
y1 148.060434167
b1 74.54148346699999
y1 74.533855317
b1 50.03008113366666
y1 50.024995700333335
b1 37.774379966999994
y1 37.770565892
b1 129.102239467
y1 148.060434167
b1 65.054757967
y1 74.533855317
b1 43.705597467
y1 50.024995700333335
b1 33.031017217
y1 37.770565892
b1 100.075690467
b2 213.159754467
b3 326.243818467
y1 98.060041167
y2 211.14410516699996
y3 324.228169167
b1 50.541483467
b2 107.083515467
b3 163.625547467
y1 49.53365881

b2 63.531115717
y1 34.266653391999995
y2 56.024660391999994
b1 98.060040467
y1 148.060434167
b1 49.533658466999995
y1 74.533855317
b1 33.358197800333336
y1 50.024995700333335
b1 157.108387467
b2 254.161151467
b3 382.256114467
b4 479.308878467
b5 576.361642467
y1 157.108388167
y2 254.161152167
y3 351.213916167
y4 479.308879167
y5 576.361643167
b1 79.057831967
b2 127.584213967
b3 191.631695467
b4 240.158077467
b5 288.684459467
y1 79.05783231699999
y2 127.584214317
y3 176.110596317
y4 240.158077817
y5 288.684459817
b1 53.040980133666665
b2 85.39190146700001
b3 128.090222467
b4 160.44114380033332
b5 192.79206513366668
y1 53.040980366999996
y2 85.39190170033334
y3 117.74282303366665
y4 160.44114403366666
y5 192.792065367
b1 40.032554217
b2 64.295745217
b3 96.31948596699999
b4 120.582676967
b5 144.845867967
y1 40.032554391999994
y2 64.295745392
y3 88.55893639199999
y4 120.582677142
y5 144.845868142
b1 115.050203467
y1 98.060041167
b1 58.028739967
y1 49.533658816999996
b1 39.021585467
y1 33.3

b2 186.123703467
b3 323.182615467
y1 100.075691167
y2 237.13460316699997
y3 365.229566167
b1 29.518008467
b2 93.565489967
b3 162.094945967
y1 50.541483817
y2 119.07093981699998
y3 183.11842131699999
b1 20.014431133666665
b2 62.71275213366667
b3 108.39905613366666
y1 34.030081367
y2 79.71638536699999
y3 122.414706367
b1 115.050203467
y1 115.050204167
b1 58.028739967
y1 58.028740317
b1 72.044390467
b2 200.139353467
b3 313.223417467
y1 114.091341167
y2 227.175405167
y3 355.270368167
b1 36.525833467
b2 100.573314967
b3 157.115346967
y1 57.549308817
y2 114.091340817
y3 178.138822317
b1 24.686314467000003
b2 67.384635467
b3 105.079323467
y1 38.701964700333335
y2 76.39665270033333
y3 119.09497370033334
b1 18.766554967
b2 50.790295717
b3 79.061311717
y1 29.278292642
y2 57.549308642
y3 89.573049392
b1 98.060040467
b2 211.14410446699998
b3 298.176132467
b4 397.244546467
y1 148.060434167
y2 247.12884816700003
y3 334.160876167
y4 447.24494016700004
b1 49.533658466999995
b2 106.07569046699999
b3 14

b1 58.028740467
b2 129.065854467
b3 242.14991846700002
y1 114.091341167
y2 227.175405167
y3 298.212519167
b1 29.518008467
b2 65.036565467
b3 121.57859746700001
y1 57.549308817
y2 114.091340817
y3 149.609897817
b1 20.014431133666665
b2 43.69346913366667
b3 81.38815713366667
y1 38.701964700333335
y2 76.39665270033333
y3 100.07569070033333
b1 15.262642467000001
b2 33.021920967
b3 61.292936967
y1 29.278292642
y2 57.549308642
y3 75.308587142
b1 12.411569267
b2 26.618992067000004
b3 49.235804867000006
y1 23.624089407
y2 46.240902207000005
y3 60.448325006999994
b1 138.066188467
b2 298.0968371906
y1 134.044784167
y2 294.0754328906
b1 69.536732467
b2 149.5520568288
y1 67.526030317
y2 147.5413546788
b1 46.693580467
b2 100.03713004153333
y1 45.353112366999994
y2 98.69666194153332
b1 129.065854467
b2 285.166965467
y1 102.054956167
y2 258.156067167
b1 65.036565467
b2 143.087120967
y1 51.531116317
y2 129.581671817
b1 43.69346913366667
b2 95.72717280033334
y1 34.689836367000005
y2 86.72354003366667
b

y1 65.054758317
y2 122.076221817
b1 38.701964467
b2 76.716273467
y1 43.70559770033333
y2 81.71990670033334
b1 29.278292467
b2 57.789024217
y1 33.031017391999995
y2 61.541749142
b1 114.091340467
b2 171.112804467
b3 327.213915467
y1 115.050204167
y2 271.151315167
y3 328.172779167
b1 57.549308466999996
b2 86.060040467
b3 164.110595967
y1 58.028740317
y2 136.079295817
y3 164.590027817
b1 38.701964467
b2 57.70911913366666
b3 109.74282280033333
y1 39.02158570033334
y2 91.055289367
y3 110.06244403366667
b1 29.278292467
b2 43.533658466999995
b3 82.558936217
y1 29.518008392000002
y2 68.543286142
y3 82.798652142
b1 138.066188467
b2 269.10667346699995
b3 356.1387014669999
b4 455.20711546699994
b5 526.244229467
y1 129.102240167
y2 200.13935416700002
y3 299.207768167
y4 386.239796167
y5 517.280281167
b1 69.536732467
b2 135.05697496699997
b3 178.57298896699996
b4 228.10719596699997
b5 263.625752967
y1 65.054758317
y2 100.57331531700001
y3 150.107522317
y4 193.623536317
y5 259.143778817
b1 46.6935804

In [12]:
print(len(test['masses_raw']))

703


In [13]:
# experiment files 
mzML = 'HEK293T_De_Novo_061122_Glu-C_B_BP_anyLength_HCD30.mzML'
log = 'App-2022-06-12_22-28-53.log'
realtime = 'HEK293T_De_Novo_061122_Glu-C_B_BP_anyLength_HCD30_realtimesearch.tsv'

In [14]:
df4, series4 = formatData(mzML, log, realtime)

num MS3s:  681
Scans match up after taking rounding discrepencies into consideration
b1 98.060040467
b2 235.11895246699999
y1 148.060434167
y2 285.11934616699995
b1 49.533658466999995
b2 118.06311446699999
y1 74.533855317
y2 143.06331131699997
b1 33.358197800333336
b2 79.04450180033332
y1 50.024995700333335
y2 95.71129970033331
b1 25.270467467
b2 59.535195466999994
y1 37.770565892
y2 72.03529389199998
b1 88.039304467
b2 187.10771846699998
b3 258.144832467
y1 114.091341167
y2 185.12845516699997
y3 284.196869167
b1 44.523290466999995
b2 94.05749746699999
b3 129.576054467
y1 57.549308817
y2 93.06786581699998
y3 142.602072817
b1 30.017952467
b2 63.04075713366666
b3 86.71979513366666
y1 38.701964700333335
y2 62.38100270033332
y3 95.403807367
b1 22.765283467
b2 47.532386966999994
b3 65.291665467
y1 29.278292642
y2 47.03757114199999
y3 71.804674642
b1 18.413682067
b2 38.227364867
b3 52.434787667
y1 23.624089407
y2 37.831512206999996
y3 57.645195007
b1 114.091340467
b2 242.14991846700002
b3 32

b4 141.43024480033333
b5 184.116437467
y1 45.353112366999994
y2 88.03930503366666
y3 121.06210970033334
y4 153.41303103366667
y5 196.1113520336667
b1 25.270467467
b2 57.294208217
b3 81.557399217
b4 106.324502717
b5 138.339147217
y1 34.266653391999995
y2 66.281297892
y3 91.048401392
y4 115.31159239200001
y5 147.33533314200002
b1 98.060040467
b2 155.081504467
y1 148.060434167
y2 205.08189816700002
b1 49.533658466999995
b2 78.044390467
y1 74.533855317
y2 103.04458731700001
b1 100.075690467
b2 214.118617467
y1 148.060434167
y2 262.103361167
b1 50.541483467
b2 107.562946967
y1 74.533855317
y2 131.555318817
b1 34.03008113366667
b2 72.04439013366667
y1 50.024995700333335
y2 88.03930470033333
b1 98.060040467
b2 155.081504467
y1 148.060434167
y2 205.08189816700002
b1 49.533658466999995
b2 78.044390467
y1 74.533855317
y2 103.04458731700001
b1 102.054955467
y1 72.044391167
b1 51.531115967
y1 36.525833817
b1 88.039304467
y1 148.060434167
b1 44.523290466999995
y1 74.533855317
b1 30.017952467
y1 50.

b1 20.014431133666665
y1 30.017952700333336
b1 161.0379251906
y1 148.075691167
b1 81.0226008288
y1 74.541483817
b1 54.35082604153333
y1 50.030081367
b1 100.075690467
b2 187.10771846699998
b3 350.171047467
b4 506.272158467
y1 148.060434167
y2 304.161545167
y3 467.224874167
y4 554.256902167
b1 50.541483467
b2 94.05749746699999
b3 175.589161967
b4 253.639717467
y1 74.533855317
y2 152.584410817
y3 234.116075317
y4 277.632089317
b1 34.03008113366667
b2 63.04075713366666
b3 117.39520013366666
b4 169.42890380033333
y1 50.024995700333335
y2 102.05869936699999
y3 156.413142367
y4 185.423818367
b1 25.774379967
b2 47.532386966999994
b3 88.298219217
b4 127.323496967
y1 37.770565892
y2 76.795843642
y3 117.561675892
y4 139.319682892
b1 72.044390467
b2 159.076418467
b3 216.097882467
y1 129.102240167
y2 186.12370416700003
y3 273.155732167
b1 36.525833467
b2 80.041847467
b3 108.552579467
y1 65.054758317
y2 93.56549031700001
y3 137.081504317
b1 24.686314467000003
b2 53.696990467
b3 72.70414513366667
y1 

y2 131.075887317
b1 129.065854467
y1 88.039305167
b1 65.036565467
y1 44.523290816999996
b1 43.69346913366667
y1 30.017952700333336
b1 33.021920967
y1 22.765283642
b1 72.044390467
b2 143.081504467
b3 256.165568467
y1 148.060434167
y2 261.144498167
y3 332.18161216699997
b1 36.525833467
b2 72.044390467
b3 128.586422467
y1 74.533855317
y2 131.075887317
y3 166.59444431699998
b1 24.686314467000003
b2 48.365352467
b3 86.06004046700001
y1 50.024995700333335
y2 87.71968370033333
y3 111.39872170033333
b1 18.766554967
b2 36.525833467
b3 64.796849467
y1 37.770565892
y2 66.041581892
y3 83.80086039199999
b1 132.047761467
y1 148.060434167
b1 66.52751896699999
y1 74.533855317
b1 129.102239467
y1 148.060434167
b1 65.054757967
y1 74.533855317
b1 43.705597467
y1 50.024995700333335
b1 114.091340467
b2 270.192451467
y1 134.044784167
y2 290.145895167
b1 57.549308466999996
b2 135.599863967
y1 67.526030317
y2 145.576585817
b1 38.701964467
b2 90.73566813366666
y1 45.353112366999994
y2 97.38681603366666
b1 29.2

b2 251.15025246699997
y1 148.060434167
y2 261.144498167
b1 69.536732467
b2 126.07876446699998
y1 74.533855317
y2 131.075887317
b1 46.693580467
b2 84.38826846699999
y1 50.024995700333335
y2 87.71968370033333
b1 164.070605467
y1 148.060434167
b1 82.538940967
y1 74.533855317
b1 55.361719467
y1 50.024995700333335
b1 98.060040467
b2 212.10296746699998
y1 148.060434167
y2 262.103361167
b1 49.533658466999995
b2 106.55512196699999
y1 74.533855317
y2 131.555318817
b1 88.039304467
y1 148.060434167
b1 44.523290466999995
y1 74.533855317
b1 30.017952467
y1 50.024995700333335
b1 22.765283467
y1 37.770565892
b1 100.075690467
b2 187.10771846699998
b3 288.155397467
y1 164.070606167
y2 265.118285167
y3 352.15031316700004
b1 50.541483467
b2 94.05749746699999
b3 144.581336967
y1 82.538941317
y2 133.062780817
y3 176.57879481700002
b1 138.066188467
b2 266.124766467
b3 323.14623046699995
b4 394.183344467
y1 129.065855167
y2 200.10296916700003
y3 257.124433167
y4 385.183011167
b1 69.536732467
b2 133.566021466

...

y1 50.024995700333335
b1 72.044390467
y1 164.070606167
b1 36.525833467
y1 82.538941317
b1 24.686314467000003
y1 55.36171970033333
b1 88.039304467
b2 216.134267467
y1 148.060434167
y2 276.155397167
b1 44.523290466999995
b2 108.570771967
y1 74.533855317
y2 138.581336817
b1 30.017952467
b2 72.716273467
y1 50.024995700333335
y2 92.72331670033333
b1 157.108387467
b2 244.140415467
b3 331.172443467
b4 478.24085746699996
y1 164.070606167
y2 311.139020167
y3 398.171048167
y4 485.20307616699995
b1 79.057831967
b2 122.573845967
b3 166.089859967
b4 239.62406696699998
y1 82.538941317
y2 156.073148317
y3 199.589162317
y4 243.10517631699997
b1 53.040980133666665
b2 82.05165613366667
b3 111.06233213366666
b4 160.08513680033332
y1 55.36171970033333
y2 104.384524367
y3 133.395200367
y4 162.405876367
b1 40.032554217
b2 61.790561217
b3 83.548568217
b4 120.31567171699999
y1 41.773108891999996
y2 78.540212392
y3 100.298219392
y4 122.05622639199999
b1 58.028740467
b2 115.050204467
b3 172.071668467
b4 269.124

b4 373.171775467
b5 430.19323946699996
y1 148.060434167
y2 205.08189816700002
y3 306.129577167
y4 393.161605167
y5 480.193633167
b1 49.533658466999995
b2 93.049672467
b3 136.56568646699998
b4 187.089525967
b5 215.60025796699998
y1 74.533855317
y2 103.04458731700001
y3 153.568426817
y4 197.084440817
y5 240.60045481699999
b1 88.039304467
b2 216.134267467
y1 164.070606167
y2 292.165569167
b1 44.523290466999995
b2 108.570771967
y1 82.538941317
y2 146.586422817
b1 30.017952467
b2 72.716273467
y1 55.36171970033333
y2 98.06004070033333
b1 22.765283467
b2 54.789024217
y1 41.773108891999996
y2 73.796849642
b1 114.091340467
b2 213.159754467
y1 138.066189167
y2 237.13460316699997
b1 57.549308466999996
b2 107.083515467
y1 69.536732817
y2 119.07093981699998
b1 38.701964467
b2 71.72476913366667
y1 46.693580700333335
y2 79.71638536699999
b1 114.091340467
b2 213.159754467
y1 138.066189167
y2 237.13460316699997
b1 57.549308466999996
b2 107.083515467
y1 69.536732817
y2 119.07093981699998
b1 38.701964467

b1 18.766554967
b2 33.021920967
y1 18.766555142
y2 33.021921142
b1 15.214699267
b2 26.618992067000004
y1 15.214699407000001
y2 26.618992207
b1 72.044390467
b2 129.065854467
y1 72.044391167
y2 129.065855167
b1 36.525833467
b2 65.036565467
y1 36.525833817
y2 65.036565817
b1 24.686314467000003
b2 43.69346913366667
y1 24.686314700333337
y2 43.693469367
b1 18.766554967
b2 33.021920967
y1 18.766555142
y2 33.021921142
b1 15.214699267
b2 26.618992067000004
y1 15.214699407000001
y2 26.618992207
b1 98.060040467
b2 226.118618467
b3 354.17719646700004
b4 441.209224467
y1 148.060434167
y2 235.092462167
y3 363.15104016699996
y4 491.209618167
b1 49.533658466999995
b2 113.562947467
b3 177.59223646700002
b4 221.108250467
y1 74.533855317
y2 118.049869317
y3 182.07915831699998
y4 246.10844731699999
b1 33.358197800333336
b2 76.044390467
b3 118.73058313366668
b4 147.74125913366666
y1 50.024995700333335
y2 79.03567170033334
y3 121.72186436699998
y4 164.40805703366667
b1 25.270467467
b2 57.285111967
b3 89.29

y2 67.71264070033332
y3 105.72694970033332
y4 143.42163770033332
y5 186.11995870033334
b1 157.108387467
b2 254.161151467
b3 385.20163646699996
y1 88.039305167
y2 219.079790167
y3 316.13255416699997
b1 79.057831967
b2 127.584213967
b3 193.10445646699998
y1 44.523290816999996
y2 110.043533317
y3 158.56991531699998
b1 53.040980133666665
b2 85.39190146700001
b3 129.07206313366666
y1 30.017952700333336
y2 73.698114367
y3 106.04903570033332
b1 40.032554217
b2 64.295745217
b3 97.05586646699999
y1 22.765283642
y2 55.525404892
y3 79.78859589199999
b1 114.091340467
y1 148.060434167
b1 57.549308466999996
y1 74.533855317
b1 38.701964467
y1 50.024995700333335
b1 98.060040467
b2 212.10296746699998
b3 325.187031467
y1 148.060434167
y2 261.144498167
y3 375.187425167
b1 49.533658466999995
b2 106.55512196699999
b3 163.097153967
y1 74.533855317
y2 131.075887317
y3 188.097350817
b1 33.358197800333336
b2 71.37250680033333
b3 109.06719480033333
y1 50.024995700333335
y2 87.71968370033333
y3 125.7339927003333

y2 235.11895316699997
y3 348.203017167
b1 57.549308466999996
b2 114.091340467
b3 162.617722467
y1 69.536732817
y2 118.06311481699998
y3 174.605146817
b1 38.701964467
b2 76.396652467
b3 108.74757380033333
y1 46.693580700333335
y2 79.04450203366666
y3 116.73919003366666


In [15]:
# experiment files 
mzML = 'HEK293T_De_Novo_061122_Glu-C_B_BP_anyLength_HCD35.mzML'
log = 'App-2022-06-11_21-20-46.log'
realtime = 'HEK293T_De_Novo_061122_Glu-C_B_BP_anyLength_HCD35_realtimesearch.tsv'

In [16]:
df5, series5 = formatData(mzML, log, realtime)

num MS3s:  1430
Scans match up after taking rounding discrepencies into consideration
b1 138.066188467
y1 148.060434167
b1 69.536732467
y1 74.533855317
b1 46.693580467
y1 50.024995700333335
b1 114.091340467
b2 213.159754467
b3 327.202681467
b4 424.25544546699996
y1 115.050204167
y2 212.10296816699997
y3 326.145895167
y4 425.214309167
b1 57.549308466999996
b2 107.083515467
b3 164.104978967
b4 212.63136096699998
y1 58.028740317
y2 106.55512231699998
y3 163.576585817
y4 213.110792817
b1 72.044390467
b2 200.102968467
y1 58.028741167
y2 186.08731916700003
b1 36.525833467
b2 100.555122467
y1 29.518008817000002
y2 93.54729781700001
b1 24.686314467000003
b2 67.37250713366667
y1 20.014431367
y2 62.70062403366668
b1 100.075690467
b2 228.17065346700002
b3 356.22923146700003
y1 148.060434167
y2 276.119012167
y3 404.213975167
b1 50.541483467
b2 114.58896496700001
b3 178.61825396700002
y1 74.533855317
y2 138.563144317
y3 202.610625817
b1 34.03008113366667
b2 76.72840213366668
b3 119.41459480033335
y

b1 129.102239467
b2 228.17065346700002
b3 325.223417467
b4 424.291831467
y1 148.060434167
y2 247.12884816700003
y3 344.18161216699997
y4 443.250026167
b1 65.054757967
b2 114.58896496700001
b3 163.115346967
b4 212.649553967
y1 74.533855317
y2 124.06806231700001
y3 172.59444431699998
y4 222.128651317
b1 43.705597467
b2 76.72840213366668
b3 109.079323467
b4 142.10212813366667
y1 50.024995700333335
y2 83.04780036700001
y3 115.39872170033333
y4 148.42152636699998
b1 72.044390467
b2 185.12845446699998
y1 148.060434167
y2 261.144498167
b1 36.525833467
b2 93.06786546699999
y1 74.533855317
y2 131.075887317
b1 24.686314467000003
b2 62.381002466999995
y1 50.024995700333335
y2 87.71968370033333
b1 18.766554967
b2 47.03757096699999
y1 37.770565892
y2 66.041581892
b1 15.214699267
b2 37.831512067
y1 30.417908007000005
y2 53.034720807
b1 98.060040467
b2 155.081504467
y1 148.060434167
y2 205.08189816700002
b1 49.533658466999995
b2 78.044390467
y1 74.533855317
y2 103.04458731700001
b1 132.047761467
b2 2

b3 171.616045467
b4 221.150252467
y1 74.541483817
y2 124.07569081699998
y3 188.12317231699998
y4 216.63390431699997
b1 53.040980133666665
b2 72.04813480033333
b3 114.74645580033332
b4 147.769260467
y1 50.030081367
y2 83.05288603366665
y3 125.75120703366666
y4 144.75836170033332
b1 40.032554217
b2 54.287920217
b3 86.311660967
b4 111.078764467
y1 37.774380142
y2 62.54148364199999
y3 94.56522439199999
y4 108.82059039199999
b1 98.060040467
y1 148.060434167
b1 49.533658466999995
y1 74.533855317
b1 114.091340467
b2 213.159754467
b3 341.254717467
y1 129.065855167
y2 257.160818167
y3 356.229232167
b1 57.549308466999996
b2 107.083515467
b3 171.130996967
y1 65.036565817
y2 129.084047317
y3 178.618254317
b1 38.701964467
b2 71.72476913366667
b3 114.42309013366666
y1 43.693469367
y2 86.391790367
y3 119.41459503366667
b1 29.278292467
b2 54.045395967
b3 86.06913671699999
y1 33.021921142
y2 65.045661892
y3 89.812765392
b1 23.624089267
b2 43.437772067000004
b3 69.056764667
y1 26.618992207
y2 52.2379848

b1 129.102239467
b2 186.123703467
b3 314.21866646700005
y1 115.050204167
y2 243.145167167
y3 300.166631167
b1 65.054757967
b2 93.565489967
b3 157.61297146700002
y1 58.028740317
y2 122.076221817
y3 150.586953817
b1 43.705597467
b2 62.71275213366667
b3 105.41107313366668
y1 39.02158570033334
y2 81.71990670033334
y3 100.72706136699999
b1 72.044390467
b2 169.097154467
y1 88.039305167
y2 185.09206916699998
b1 36.525833467
b2 85.052215467
y1 44.523290816999996
y2 93.04967281699999
b1 24.686314467000003
b2 57.03723580033333
y1 30.017952700333336
y2 62.36887403366666
b1 18.766554967
b2 43.029745967
y1 22.765283642
y2 47.02847464199999
b1 98.060040467
y1 148.060434167
b1 49.533658466999995
y1 74.533855317
b1 33.358197800333336
y1 50.024995700333335
b1 25.270467467
y1 37.770565892
b1 138.066188467
b2 235.11895246699999
y1 148.060434167
y2 245.113198167
b1 69.536732467
b2 118.06311446699999
y1 74.533855317
y2 123.060237317
b1 46.693580467
b2 79.04450180033332
y1 50.024995700333335
y2 82.375917033

y1 58.028741167
b1 81.0226008288
y1 29.518008817000002
b1 54.35082604153333
y1 20.014431367
b1 100.075690467
b2 201.123369467
b3 300.191783467
b4 399.260197467
y1 175.118952167
y2 274.187366167
y3 373.255780167
y4 474.303459167
b1 50.541483467
b2 101.065322967
b3 150.599529967
b4 200.133736967
y1 88.063114317
y2 137.597321317
y3 187.131528317
y4 237.655367817
b1 34.03008113366667
b2 67.71264080033333
b3 100.73544546699999
b4 133.75825013366668
y1 59.044501700333335
y2 92.067306367
y3 125.09011103366667
y4 158.77267070033335
b1 100.075690467
b2 187.10771846699998
b3 350.171047467
b4 506.272158467
y1 148.060434167
y2 304.161545167
y3 467.224874167
y4 554.256902167
b1 50.541483467
b2 94.05749746699999
b3 175.589161967
b4 253.639717467
y1 74.533855317
y2 152.584410817
y3 234.116075317
y4 277.632089317
b1 34.03008113366667
b2 63.04075713366666
b3 117.39520013366666
b4 169.42890380033333
y1 50.024995700333335
y2 102.05869936699999
y3 156.413142367
y4 185.423818367
b1 25.774379967
b2 47.53238

...

b3 89.317948967
b4 114.57986871700001
y1 37.770565892
y2 63.032485642000005
y3 95.056226392
y4 127.079967142
b1 98.060040467
b2 226.155003467
b3 354.24996646700004
b4 455.29764546700005
y1 148.060434167
y2 249.10811316700003
y3 377.203076167
y4 505.298039167
b1 49.533658466999995
b2 113.581139967
b3 177.62862146700002
b4 228.15246096700002
y1 74.533855317
y2 125.05769481700001
y3 189.105176317
y4 253.152657817
b1 33.358197800333336
b2 76.05651880033334
b3 118.75483980033334
b4 152.437399467
y1 50.024995700333335
y2 83.70755536700001
y3 126.405876367
y4 169.104197367
b1 25.270467467
b2 57.294208217
b3 89.317948967
b4 114.57986871700001
y1 37.770565892
y2 63.032485642000005
y3 95.056226392
y4 127.079967142
b1 129.102239467
b2 257.160817467
y1 114.091341167
y2 242.149919167
b1 65.054757967
b2 129.084046967
y1 57.549308817
y2 121.578597817
b1 43.705597467
b2 86.39179013366667
y1 38.701964700333335
y2 81.388157367
b1 33.031017217
b2 65.045661717
y1 29.278292642
y2 61.292937142
b1 26.6262690

y5 180.76780836699996
b1 148.075690467
y1 148.060434167
b1 74.54148346699999
y1 74.533855317
b1 50.03008113366666
y1 50.024995700333335
b1 88.039304467
y1 138.066189167
b1 44.523290466999995
y1 69.536732817
b1 30.017952467
y1 46.693580700333335
b1 22.765283467
y1 35.272004642
b1 18.413682067
y1 28.419059007000005
b1 114.091340467
b2 228.134267467
y1 129.102240167
y2 243.145167167
b1 57.549308466999996
b2 114.570771967
y1 65.054758317
y2 122.076221817
b1 38.701964467
b2 76.716273467
y1 43.70559770033333
y2 81.71990670033334
b1 29.278292467
b2 57.789024217
y1 33.031017391999995
y2 61.541749142
b1 88.039304467
b2 185.092068467
y1 72.044391167
y2 169.09715516699998
b1 44.523290466999995
b2 93.049672467
y1 36.525833817
y2 85.05221581699999
b1 30.017952467
b2 62.368873800333326
y1 24.686314700333337
y2 57.03723603366666
b1 22.765283467
b2 47.028474466999995
y1 18.766555142
y2 43.02974614199999
b1 18.413682067
b2 37.824234867
y1 15.214699407000001
y2 34.625252207
b1 148.075690467
b2 249.12336

b4 89.53894358759999
y1 23.85042858366667
y2 50.52220337093333
y3 75.03360570426666
y4 97.87675770426665
b1 138.066188467
b2 252.10911546699998
y1 148.060434167
y2 262.103361167
b1 69.536732467
b2 126.55819596699999
y1 74.533855317
y2 131.555318817
b1 46.693580467
b2 84.70788946699999
y1 50.024995700333335
y2 88.03930470033333
b1 35.272004466999995
b2 63.78273621699999
y1 37.770565892
y2 66.281297642
b1 28.419058867
b2 51.227644266999995
y1 30.417908007000005
y2 53.226493407
b1 58.028740467
b2 187.07133346700002
b3 284.124097467
y1 102.054956167
y2 199.10772016699997
y3 328.150313167
b1 29.518008467
b2 94.039304967
b3 142.565686967
y1 51.531116317
y2 100.05749831699998
y3 164.578794817
b1 20.014431133666665
b2 63.02862880033334
b3 95.37955013366667
y1 34.689836367000005
y2 67.04075770033332
y3 110.05495536699999
b1 15.262642467000001
b2 47.523290717
b3 71.786481717
y1 26.269196392
y2 50.53238739199999
y3 82.79303564199999
b1 114.091340467
b2 213.159754467
y1 100.075691167
y2 199.144105

y2 276.155397167
y3 423.22381116699995
y4 480.24527516699993
b1 49.533658466999995
b2 78.044390467
b3 151.578597467
b4 215.626078967
y1 74.533855317
y2 138.581336817
y3 212.11554381699997
y4 240.62627581699996
b1 98.060040467
b2 155.081504467
b3 302.149918467
b4 430.244881467
y1 148.060434167
y2 276.155397167
y3 423.22381116699995
y4 480.24527516699993
b1 49.533658466999995
b2 78.044390467
b3 151.578597467
b4 215.626078967
y1 74.533855317
y2 138.581336817
y3 212.11554381699997
y4 240.62627581699996
b1 115.050203467
b2 216.097882467
b3 303.129910467
y1 148.060434167
y2 235.092462167
y3 336.14014116699997
b1 58.028739967
b2 108.552579467
b3 152.068593467
y1 74.533855317
y2 118.049869317
y3 168.57370881699998
b1 39.021585467
b2 72.70414513366667
b3 101.71482113366666
y1 50.024995700333335
y2 79.03567170033334
y3 112.71823136699999
b1 88.039304467
b2 201.12336846699998
b3 329.181946467
b4 426.234710467
y1 116.034220167
y2 213.08698416699997
y3 341.145562167
y4 454.229626167
b1 44.523290466

b1 43.705597467
b2 77.38815713366667
y1 50.024995700333335
y2 83.70755536700001
b1 33.031017217
b2 58.292936967
y1 37.770565892
y2 63.032485642000005
b1 26.626269067000003
b2 46.83580486700001
y1 30.417908007000005
y2 50.627443807000006
b1 114.091340467
b2 251.15025246699997
y1 134.044784167
y2 271.10369616699995
b1 57.549308466999996
b2 126.07876446699998
y1 67.526030317
y2 136.05548631699997
b1 38.701964467
b2 84.38826846699999
y1 45.353112366999994
y2 91.03941636699999
b1 29.278292467
b2 63.54302046699999
y1 34.266653391999995
y2 68.53138139199999
b1 114.091340467
b2 185.12845446699998
y1 129.065855167
y2 200.10296916700003
b1 57.549308466999996
b2 93.06786546699999
y1 65.036565817
y2 100.55512281700001
b1 102.054955467
y1 129.102240167
b1 51.531115967
y1 65.054758317
b1 34.68983613366667
y1 43.70559770033333
b1 129.102239467
b2 228.17065346700002
b3 375.23906746700004
b4 488.32313146700005
y1 72.044391167
y2 185.12845516699997
y3 332.19686916699993
y4 431.26528316699995
b1 65.05475

In [17]:
print(len(test['masses_raw']))

1294


In [18]:
# experiment files 
mzML = 'HEK293T_De_Novo_061122_Glu-C_B_BP_anyLength2_HCD35.mzML'
log = 'App-2022-06-12_19-59-11.log'
realtime = 'HEK293T_De_Novo_061122_Glu-C_B_BP_anyLength2_HCD35_realtimesearch.tsv'

In [19]:
df6, series6 = formatData(mzML, log, realtime)

num MS3s:  1114
Scans match up after taking rounding discrepencies into consideration
b1 100.075690467
b2 231.116175467
b3 330.184589467
b4 461.225074467
b5 558.277838467
y1 115.050204167
y2 212.10296816699997
y3 343.14345316699996
y4 442.211867167
y5 573.252352167
b1 50.541483467
b2 116.061725967
b3 165.595932967
b4 231.116175467
b5 279.642557467
y1 58.028740317
y2 106.55512231699998
y3 172.07536481699998
y4 221.609571817
y5 287.129814317
b1 34.03008113366667
b2 77.71024280033333
b3 110.733047467
b4 154.41320913366667
b5 186.764130467
y1 39.02158570033334
y2 71.37250703366665
y3 115.05266870033331
y4 148.075473367
y5 191.75563503366666
b1 25.774379967
b2 58.534501217
b3 83.301604717
b4 116.061725967
b5 140.324916967
y1 29.518008392000002
y2 53.78119939199999
y3 86.54132064199999
y4 111.30842414199999
y5 144.068545392
b1 20.820959267
b2 47.029056267
b3 66.842739067
b4 93.050836067
b5 112.46138886700001
y1 23.815862007
y2 43.226414807
y3 69.43451180699999
y4 89.24819460699999
y5 115.456

y2 124.068062317
y3 197.60226931699998
y4 226.11300131699997
y5 294.642457317
b1 38.701964467
b2 84.38826846699999
b3 103.39542313366665
b4 152.4182278003333
b5 190.11291580033333
y1 45.353112366999994
y2 83.047800367
y3 132.07060503366665
y4 151.0777597003333
y5 196.7640637003333
b1 29.278292467
b2 63.54302046699999
b3 77.79838646699999
b4 114.56548996699999
b5 142.836505967
y1 34.266653391999995
y2 62.537669392
y3 99.30477289199999
y4 113.56013889199998
y5 147.824866892
b1 23.624089267
b2 51.035871666999995
b3 62.44016446699999
b4 91.85384726699999
b5 114.470660067
y1 27.614778007
y2 50.231590807
y3 79.645273607
y4 91.04956640699999
y5 118.461348807
b1 100.075690467
b2 228.13426846700003
y1 72.044391167
y2 200.10296916700003
b1 50.541483467
b2 114.57077246700001
y1 36.525833817
y2 100.55512281700001
b1 115.050203467
b2 214.118617467
b3 400.197930467
b4 457.219394467
y1 100.075691167
y2 157.09715516699998
y3 343.176468167
y4 442.244882167
b1 58.028739967
b2 107.562946967
b3 200.602603

y1 164.070606167
y2 277.154670167
y3 390.238734167
b1 50.541483467
b2 107.083515467
b3 163.625547467
y1 82.538941317
y2 139.080973317
y3 195.623005317
b1 34.03008113366667
b2 71.72476913366667
b3 109.41945713366665
y1 55.36171970033333
y2 93.05640770033334
y3 130.75109570033334
b1 25.774379967
b2 54.045395967
b3 82.316411967
y1 41.773108891999996
y2 70.044124892
y3 98.315140892
b1 129.102239467
b2 242.18630346700002
b3 428.26561646700003
y1 138.066189167
y2 324.145502167
y3 437.22956616700003
b1 65.054757967
b2 121.596789967
b3 214.636446467
y1 69.536732817
y2 162.576389317
y3 219.118421317
b1 157.108387467
b2 254.161151467
b3 401.229565467
b4 488.261593467
y1 161.03792589059998
y2 248.0699538906
y3 395.13836789059997
y4 492.1911318906
b1 79.057831967
b2 127.584213967
b3 201.118420967
b4 244.634434967
y1 81.02260117879999
y2 124.5386151788
y3 198.07282217879998
y4 246.5992041788
b1 53.040980133666665
b2 85.39190146700001
b3 134.41470613366667
b4 163.42538213366666
y1 54.350826274866655

b1 114.091340467
b2 201.12336846699998
b3 288.15539646699995
b4 375.1874244669999
y1 148.060434167
y2 235.092462167
y3 322.124490167
y4 409.156518167
b1 57.549308466999996
b2 101.06532246699999
b3 144.58133646699997
b4 188.09735046699996
y1 74.533855317
y2 118.049869317
y3 161.565883317
y4 205.081897317
b1 38.701964467
b2 67.71264046699999
b3 96.72331646699998
b4 125.73399246699996
y1 50.024995700333335
y2 79.03567170033334
y3 108.04634770033333
y4 137.05702370033333
b1 115.050203467
b2 252.10911546699998
b3 349.16187946699995
b4 486.2207914669999
b5 599.3048554669999
y1 98.060041167
y2 211.14410516699996
y3 348.203017167
y4 445.255781167
y5 582.314693167
b1 58.028739967
b2 126.55819596699999
b3 175.08457796699997
b4 243.61403396699995
b5 300.15606596699996
y1 49.533658816999996
y2 106.07569081699998
y3 174.605146817
y4 223.131528817
y5 291.660984817
b1 39.021585467
b2 84.70788946699999
b3 117.05881080033332
b4 162.7451148003333
b5 200.4398028003333
y1 33.35819803366667
y2 71.052886033

b1 138.066188467
y1 148.060434167
b1 69.536732467
y1 74.533855317
b1 46.693580467
y1 50.024995700333335
b1 157.108387467
y1 102.054956167
b1 79.057831967
y1 51.531116317
b1 53.040980133666665
y1 34.689836367000005
b1 40.032554217
y1 26.269196392
b1 72.044390467
b2 186.087317467
b3 287.134996467
b4 400.219060467
b5 556.320171467
y1 134.044784167
y2 290.145895167
y3 403.229959167
y4 504.277638167
y5 618.320565167
b1 36.525833467
b2 93.547296967
b3 144.071136467
b4 200.613168467
b5 278.663723967
y1 67.526030317
y2 145.576585817
y3 202.118617817
y4 252.642457317
y5 309.663920817
b1 24.686314467000003
b2 62.70062346699999
b3 96.38318313366666
b4 134.07787113366666
b5 186.11157480033333
y1 45.353112366999994
y2 97.38681603366666
y3 135.08150403366668
y4 168.76406370033334
y5 206.77837270033334
b1 129.102239467
b2 230.14991846700002
b3 377.218332467
y1 148.060434167
y2 295.12884816699994
y3 396.17652716699996
b1 65.054757967
b2 115.57859746700001
b3 189.112804467
y1 74.533855317
y2 148.068062

...

b1 148.075690467
b2 261.15975446699997
y1 115.050204167
y2 228.13426816700002
b1 74.54148346699999
b2 131.08351546699998
y1 58.028740317
y2 114.570772317
b1 115.050203467
b2 228.134267467
y1 114.091341167
y2 227.175405167
b1 58.028739967
b2 114.570771967
y1 57.549308817
y2 114.091340817
b1 39.021585467
b2 76.716273467
y1 38.701964700333335
y2 76.39665270033333
b1 88.039304467
y1 148.060434167
b1 44.523290466999995
y1 74.533855317
b1 30.017952467
y1 50.024995700333335
b1 22.765283467
y1 37.770565892
b1 18.413682067
y1 30.417908007000005
b1 148.075690467
b2 308.1063391906
b3 395.1383671906
y1 148.060434167
y2 235.092462167
y3 395.1231108906
b1 74.54148346699999
b2 154.5568078288
b3 198.0728218288
y1 74.533855317
y2 118.049869317
y3 198.0651936788
b1 50.03008113366666
b2 103.3736307082
b3 132.3843067082
y1 50.024995700333335
y2 79.03567170033334
y3 132.37922127486667
b1 37.774379966999994
b2 77.7820421479
b3 99.54004914789999
y1 37.770565892
y2 59.528572892
y3 99.5362350729
b1 129.0658544

y4 505.298039167
b1 49.533658466999995
b2 113.581139967
b3 177.62862146700002
b4 228.15246096700002
y1 74.533855317
y2 125.05769481700001
y3 189.105176317
y4 253.152657817
b1 33.358197800333336
b2 76.05651880033334
b3 118.75483980033334
b4 152.437399467
y1 50.024995700333335
y2 83.70755536700001
y3 126.405876367
y4 169.104197367
b1 25.270467467
b2 57.294208217
b3 89.317948967
b4 114.57986871700001
y1 37.770565892
y2 63.032485642000005
y3 95.056226392
y4 127.079967142
b1 98.060040467
b2 226.155003467
b3 354.24996646700004
b4 455.29764546700005
y1 148.060434167
y2 249.10811316700003
y3 377.203076167
y4 505.298039167
b1 49.533658466999995
b2 113.581139967
b3 177.62862146700002
b4 228.15246096700002
y1 74.533855317
y2 125.05769481700001
y3 189.105176317
y4 253.152657817
b1 33.358197800333336
b2 76.05651880033334
b3 118.75483980033334
b4 152.437399467
y1 50.024995700333335
y2 83.70755536700001
y3 126.405876367
y4 169.104197367
b1 25.270467467
b2 57.294208217
b3 89.317948967
b4 114.579868717

b1 58.028740467
b2 195.087652467
b3 252.109116467
y1 134.044784167
y2 191.066248167
y3 328.12516016699993
b1 29.518008467
b2 98.047464467
b3 126.558196467
y1 67.526030317
y2 96.036762317
y3 164.56621831699997
b1 20.014431133666665
b2 65.70073513366667
b3 84.70788980033333
y1 45.353112366999994
y2 64.36026703366666
y3 110.04657103366664
b1 15.262642467000001
b2 49.527370467
b3 63.782736467
y1 34.266653391999995
y2 48.522019392
y3 82.78674739199998
b1 72.044390467
b2 235.107719467
y1 148.060434167
y2 311.123763167
b1 36.525833467
b2 118.057497967
y1 74.533855317
y2 156.065519817
b1 24.686314467000003
b2 79.040757467
y1 50.024995700333335
y2 104.37943870033332
b1 18.766554967
b2 59.532387217
y1 37.770565892
y2 78.536398142
b1 102.054955467
y1 134.044784167
b1 51.531115967
y1 67.526030317
b1 34.68983613366667
y1 45.353112366999994
b1 26.269196217
y1 34.266653391999995
b1 98.060040467
b2 226.155003467
b3 354.24996646700004
b4 455.29764546700005
y1 148.060434167
y2 249.10811316700003
y3 377.

b1 74.54148346699999
b2 131.08351546699998
b3 195.112804467
y1 57.549308817
y2 121.578597817
y3 178.120629817
b1 50.03008113366666
b2 87.72476913366665
b3 130.41096180033333
y1 38.701964700333335
y2 81.388157367
y3 119.082845367
b1 37.774379966999994
b2 66.04539596699999
b3 98.060040467
y1 29.278292642
y2 61.292937142
y3 89.563953142
b1 30.420959267
b2 53.03777206699999
b3 78.64948766699999
y1 23.624089407
y2 49.235805007
y3 71.852617807
b1 114.091340467
b2 227.175404467
y1 157.108388167
y2 270.192452167
b1 57.549308466999996
b2 114.091340467
y1 79.05783231699999
y2 135.599864317
b1 38.701964467
b2 76.396652467
y1 53.040980366999996
y2 90.735668367
b1 29.278292467
b2 57.549308466999996
y1 40.032554391999994
y2 68.303570392
b1 58.028740467
b2 195.087652467
b3 351.188763467
y1 148.060434167
y2 304.161545167
y3 441.220457167
b1 29.518008467
b2 98.047464467
b3 176.098019967
y1 74.533855317
y2 152.584410817
y3 221.113866817
b1 20.014431133666665
b2 65.70073513366667
b3 117.73443880033334
y1

b1 132.047761467
b2 203.084875467
b3 290.116903467
y1 114.091341167
y2 201.12336916699996
y3 272.16048316699994
b1 66.52751896699999
b2 102.046075967
b3 145.562089967
y1 57.549308817
y2 101.06532281699998
y3 136.58387981699997
b1 44.68743813366666
b2 68.36647613366667
b3 97.37715213366666
y1 38.701964700333335
y2 67.71264070033332
y3 91.39167870033332
b1 33.767397716999994
b2 51.526676216999995
b3 73.284683217
y1 29.278292642
y2 51.03629964199999
y3 68.79557814199998
b1 148.075690467
b2 276.17065346699997
y1 102.054956167
y2 230.149919167
b1 74.54148346699999
b2 138.58896496699998
y1 51.531116317
y2 115.578597817
b1 50.03008113366666
b2 92.72840213366665
y1 34.689836367000005
y2 77.388157367
b1 37.774379966999994
b2 69.79812071699999
y1 26.269196392
y2 58.292937142
b1 30.420959267
b2 56.03995186699999
y1 21.216812407
y2 46.835805007000005
b1 157.108387467
b2 214.129851467
b3 311.182615467
y1 88.039305167
y2 185.09206916699998
y3 242.113533167
b1 79.057831967
b2 107.568563967
b3 156.094

In [20]:
# experiment files 
mzML = 'HEK293T_De_Novo_061122_Glu-C_B_BP_anyLength_HCD40.mzML'
log = 'App-2022-06-12_16-55-56.log'
realtime = 'HEK293T_De_Novo_061122_Glu-C_B_BP_anyLength_HCD40_realtimesearch.tsv'

In [21]:
df7, series7 = formatData(mzML, log, realtime)

num MS3s:  1023
Scans match up after taking rounding discrepencies into consideration
b1 129.065854467
b2 186.08731846700002
b3 301.114261467
b4 398.16702546700003
b5 455.188489467
y1 114.091341167
y2 171.11280516699998
y3 268.16556916699994
y4 383.19251216699996
y5 440.21397616699994
b1 65.036565467
b2 93.547297467
b3 151.060768967
b4 199.587150967
b5 228.097882967
y1 57.549308817
y2 86.06004081699999
y3 134.58642281699997
y4 192.09989431699998
y5 220.61062631699997
b1 43.69346913366667
b2 62.70062380033334
b3 101.04293813366667
b4 133.393859467
b5 152.40101413366668
y1 38.701964700333335
y2 57.70911936699999
y3 90.06004070033332
y4 128.40235503366665
y5 147.4095097003333
b1 33.021920967
b2 47.277286967
b3 76.034022717
b4 100.297213717
b5 114.552579717
y1 29.278292642
y2 43.53365864199999
y3 67.79684964199998
y4 96.55358539199999
y5 110.80895139199998
b1 88.039304467
b2 202.08223146699999
y1 134.044784167
y2 248.087711167
b1 44.523290466999995
b2 101.54475396699999
y1 67.526030317
y2 

y3 112.05847636699998
b1 72.044390467
y1 58.028741167
b1 36.525833467
y1 29.518008817000002
b1 24.686314467000003
y1 20.014431367
b1 18.766554967
y1 15.262642642000001
b1 72.044390467
b2 143.081504467
y1 114.091341167
y2 185.12845516699997
b1 36.525833467
b2 72.044390467
y1 57.549308817
y2 93.06786581699998
b1 24.686314467000003
b2 48.365352467
y1 38.701964700333335
y2 62.38100270033332
b1 114.091340467
b2 213.159754467
b3 270.181218467
b4 383.265282467
y1 114.091341167
y2 227.175405167
y3 284.196869167
y4 383.265283167
b1 57.549308466999996
b2 107.083515467
b3 135.594247467
b4 192.136279467
y1 57.549308817
y2 114.091340817
y3 142.602072817
y4 192.136279817
b1 38.701964467
b2 71.72476913366667
b3 90.73192380033333
b4 128.42661180033335
y1 38.701964700333335
y2 76.39665270033333
y3 95.403807367
y4 128.42661203366666
b1 29.278292467
b2 54.045395967
b3 68.300761967
b4 96.571777967
y1 29.278292642
y2 57.549308642
y3 71.804674642
y4 96.571778142
b1 157.108387467
b2 256.176801467
b3 412.2779

y2 96.71627403366666
y3 115.72342870033333
b1 33.031017217
b2 47.286383217
b3 72.548302967
y1 47.527104892
y2 72.789024642
y3 87.044390642
b1 164.070605467
b2 235.107719467
b3 348.191783467
b4 476.286746467
y1 134.044784167
y2 262.139747167
y3 375.223811167
y4 446.26092516700004
b1 82.538940967
b2 118.057497967
b3 174.599529967
b4 238.647011467
y1 67.526030317
y2 131.573511817
y3 188.115543817
y4 223.63410081700002
b1 55.361719467
b2 79.040757467
b3 116.73544546699999
b4 159.433766467
y1 45.353112366999994
y2 88.051433367
y3 125.746121367
y4 149.42515936700002
b1 41.773108717
b2 59.532387217
b3 87.803403217
b4 119.827143967
y1 34.266653391999995
y2 66.290394142
y3 94.561410142
y4 112.32068864200001
b1 100.075690467
b2 213.159754467
y1 114.091341167
y2 227.175405167
b1 50.541483467
b2 107.083515467
y1 57.549308817
y2 114.091340817
b1 34.03008113366667
b2 71.72476913366667
y1 38.701964700333335
y2 76.39665270033333
b1 25.774379967
b2 54.045395967
y1 29.278292642
y2 57.549308642
b1 20.820

y2 78.37591703366667
b1 37.774379966999994
b2 62.54148346699999
y1 34.266653391999995
y2 59.033756892
b1 88.039304467
b2 145.060768467
b3 292.12918246699996
y1 148.060434167
y2 295.12884816699994
y3 352.1503121669999
b1 44.523290466999995
b2 73.034022467
b3 146.56822946699998
y1 74.533855317
y2 148.06806231699997
y3 176.57879431699996
b1 30.017952467
b2 49.025107133666666
b3 98.04791180033332
y1 50.024995700333335
y2 99.04780036699998
y3 118.05495503366664
b1 22.765283467
b2 37.020649467
b3 73.78775296699999
y1 37.770565892
y2 74.53766939199998
y3 88.79303539199998
b1 98.060040467
b2 258.09068919059996
b3 414.19180019059996
b4 551.2507121906
b5 622.2878261906
y1 148.060434167
y2 219.097548167
y3 356.156460167
y4 512.257571167
y5 672.2882198906001
b1 49.533658466999995
b2 129.54898282879998
b3 207.59953832879998
b4 276.1289943288
b5 311.6475513288
y1 74.533855317
y2 110.052412317
y3 178.581868317
y4 256.632423817
y5 336.64774817880004
b1 33.358197800333336
b2 86.70174737486666
b3 138.73

b3 282.11968046699997
b4 339.14114446699995
y1 88.039305167
y2 145.060769167
y3 232.09279716700001
y4 369.151709167
b1 29.518008467
b2 98.047464467
b3 141.56347846699998
b4 170.07421046699997
y1 44.523290816999996
y2 73.034022817
y3 116.550036817
y4 185.079492817
b1 20.014431133666665
b2 65.70073513366667
b3 94.71141113366666
b4 113.71856580033331
y1 30.017952700333336
y2 49.025107367
y3 78.03578336700001
y4 123.72208736699999
b1 164.070605467
b2 235.107719467
y1 148.060434167
y2 219.097548167
b1 82.538940967
b2 118.057497967
y1 74.533855317
y2 110.052412317
b1 55.361719467
b2 79.040757467
y1 50.024995700333335
y2 73.70403370033334
b1 41.773108717
b2 59.532387217
y1 37.770565892
y2 55.529844392
b1 129.102239467
y1 148.060434167
b1 65.054757967
y1 74.533855317
b1 43.705597467
y1 50.024995700333335
b1 33.031017217
y1 37.770565892
b1 138.066188467
y1 98.060041167
b1 69.536732467
y1 49.533658816999996
b1 46.693580467
y1 33.35819803366667
b1 129.102239467
y1 164.070606167
b1 65.054757967
y1

...

y1 72.044391167
y2 169.09715516699998
b1 44.523290466999995
b2 93.049672467
y1 36.525833817
y2 85.05221581699999
b1 30.017952467
b2 62.368873800333326
y1 24.686314700333337
y2 57.03723603366666
b1 22.765283467
b2 47.028474466999995
y1 18.766555142
y2 43.02974614199999
b1 129.102239467
b2 242.18630346700002
y1 148.060434167
y2 261.144498167
b1 65.054757967
b2 121.596789967
y1 74.533855317
y2 131.075887317
b1 43.705597467
b2 81.400285467
y1 50.024995700333335
y2 87.71968370033333
b1 98.060040467
b2 226.155003467
b3 354.24996646700004
b4 455.29764546700005
y1 148.060434167
y2 249.10811316700003
y3 377.203076167
y4 505.298039167
b1 49.533658466999995
b2 113.581139967
b3 177.62862146700002
b4 228.15246096700002
y1 74.533855317
y2 125.05769481700001
y3 189.105176317
y4 253.152657817
b1 33.358197800333336
b2 76.05651880033334
b3 118.75483980033334
b4 152.437399467
y1 50.024995700333335
y2 83.70755536700001
y3 126.405876367
y4 169.104197367
b1 25.270467467
b2 57.294208217
b3 89.317948967
b4 11

y2 124.06806231700001
y3 167.584076317
b1 24.686314467000003
b2 53.696990467
b3 86.71979513366666
y1 50.024995700333335
y2 83.04780036700001
y3 112.05847636700001
b1 18.766554967
b2 40.524561967
b3 65.291665467
y1 37.770565892
y2 62.537669392000005
y3 84.295676392
b1 15.214699267
b2 32.621104867
b3 52.434787667
y1 30.417908007000005
y2 50.23159080700001
y3 67.637996407
b1 98.060040467
y1 148.060434167
b1 49.533658466999995
y1 74.533855317
b1 33.358197800333336
y1 50.024995700333335
b1 98.060040467
b2 226.155003467
b3 354.24996646700004
b4 455.29764546700005
y1 148.060434167
y2 249.10811316700003
y3 377.203076167
y4 505.298039167
b1 49.533658466999995
b2 113.581139967
b3 177.62862146700002
b4 228.15246096700002
y1 74.533855317
y2 125.05769481700001
y3 189.105176317
y4 253.152657817
b1 33.358197800333336
b2 76.05651880033334
b3 118.75483980033334
b4 152.437399467
y1 50.024995700333335
y2 83.70755536700001
y3 126.405876367
y4 169.104197367
b1 25.270467467
b2 57.294208217
b3 89.317948967
b

b2 105.71990680033333
b3 157.75361046700002
b4 176.7607651336667
y1 50.024995700333335
y2 69.032150367
y3 121.06585403366665
y4 163.75204670033332
b1 47.527104717
b2 79.541749217
b3 118.567026967
b4 132.822392967
y1 37.770565892
y2 52.025931892
y3 91.05120964199999
y4 123.06585414199999
b1 157.108387467
b2 270.192451467
b3 327.213915467
y1 58.028741167
y2 115.050205167
y3 228.134269167
b1 79.057831967
b2 135.599863967
b3 164.110595967
y1 29.518008817000002
y2 58.028740817
y3 114.570772817
b1 53.040980133666665
b2 90.73566813366666
b3 109.74282280033333
y1 20.014431367
y2 39.02158603366667
y3 76.71627403366666
b1 40.032554217
b2 68.303570217
b3 82.558936217
y1 15.262642642000001
y2 29.518008642
y3 57.789024642
b1 114.091340467
y1 164.070606167
b1 57.549308466999996
y1 82.538941317
b1 38.701964467
y1 55.36171970033333
b1 138.066188467
b2 266.124766467
b3 367.172445467
y1 148.060434167
y2 249.10811316700003
y3 377.166691167
b1 69.536732467
b2 133.56602146699998
b3 184.089860967
y1 74.5338

b1 23.624089267
b2 46.432674667
y1 26.626269207
y2 49.434854607000005
b1 102.054955467
b2 262.0856041906
b3 409.1540181906
y1 148.060434167
y2 295.12884816699994
y3 455.15949689059994
b1 51.531115967
b2 131.5464403288
b3 205.0806473288
y1 74.533855317
y2 148.06806231699997
y3 228.08338667879997
b1 34.68983613366667
b2 88.0333857082
b3 137.05619037486667
y1 50.024995700333335
y2 99.04780036699998
y3 152.39134994153332
b1 26.269196217
b2 66.2768583979
b3 103.0439618979
y1 37.770565892
y2 74.53766939199998
y3 114.54533157289998
b1 58.028740467
b2 171.112804467
b3 284.196868467
b4 431.265282467
b5 559.360245467
y1 161.03792589059998
y2 289.1328888906
y3 436.2013028906
y4 549.2853668906
y5 662.3694308906
b1 29.518008467
b2 86.060040467
b3 142.602072467
b4 216.136279467
b5 280.183760967
y1 81.02260117879999
y2 145.0700826788
y3 218.6042896788
y4 275.1463216788
y5 331.6883536788
b1 20.014431133666665
b2 57.70911913366666
b3 95.40380713366666
b4 144.42661180033335
b5 187.12493280033334
y1 54.3

b1 132.047761467
y1 114.091341167
b1 66.52751896699999
y1 57.549308817
b1 44.68743813366666
y1 38.701964700333335
b1 33.767397716999994
y1 29.278292642
b1 114.091340467
y1 72.044391167
b1 57.549308466999996
y1 36.525833817
b1 38.701964467
y1 24.686314700333337
b1 29.278292467
y1 18.766555142
b1 115.050203467
b2 214.118617467
y1 134.044784167
y2 233.113198167
b1 58.028739967
b2 107.562946967
y1 67.526030317
y2 117.060237317
b1 39.021585467
b2 72.04439013366667
y1 45.353112366999994
y2 78.37591703366667
b1 58.028740467
b2 205.097154467
b3 292.12918246699996
y1 134.044784167
y2 221.07681216699999
y3 368.145226167
b1 29.518008467
b2 103.052215467
b3 146.56822946699998
y1 67.526030317
y2 111.04204431699999
y3 184.576251317
b1 20.014431133666665
b2 69.03723580033333
b3 98.04791180033332
y1 45.353112366999994
y2 74.363788367
y3 123.38659303366666
b1 129.102239467
b2 230.14991846700002
y1 148.060434167
y2 249.10811316700003
b1 65.054757967
b2 115.57859746700001
y1 74.533855317
y2 125.057694817

In [22]:
print(len(test['masses_raw']))

914


In [36]:
# combine all dataframes 
all_dfs = [df1, df2, df3, df4, df5, df6, df7]
concat_df = pd.concat(all_dfs).reset_index(drop=True)

In [24]:
concat_df

,masses_raw,intensities_raw,charge,modified_sequence,mass_analyzer,collision_energy,target_fragment,matches_charge1,masses_the_charge1,masses_raw_charge1,...,masses_raw_charge4,intensities_raw_charge4,matches_charge5,masses_the_charge5,masses_raw_charge5,intensities_raw_charge5,matches_charge6,masses_the_charge6,masses_raw_charge6,intensities_raw_charge6
0,54.40834045410156 72.08840942382812 72.6154479...,1847.457 1044.1062 944.0088 927.47253 941.6198...,2,SE,FTMS,10,y2,y1,148.060434167,148.0601806640625,...,,,,,,,,,,
1,54.40696716308594 54.40972137451172 70.0648117...,1802.849 918.2612 4112.844 878.95984 1029.2969...,2,IP,FTMS,10,b2,y1,98.060041167,98.06021118164062,...,,,,,,,,,,
2,53.29237365722656 54.40982437133789 58.5242881...,848.94354 1105.8538 1257.8536 921.262 1054.482...,2,TK,FTMS,10,b2,y1,129.102240167,129.1013946533203,...,,,,,,,,,,
3,62.79367446899414 66.73774719238281 68.6735992...,910.83417 941.222 894.2407 840.0121 787.6718 8...,2,TK,FTMS,10,b2,y1,129.102240167,129.102294921875,...,,,,,,,,,,
4,50.54601287841797 54.40740966796875 54.4102096...,1046.0387 1604.429 1038.8164 902.5171 960.406 ...,2,SE,FTMS,10,y2,y1,148.060434167,148.0605010986328,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,54.408470153808594 70.06488800048828 70.590484...,3260.9204 6078.789 2339.1902 10882.506 2424.62...,4,PKKTE,FTMS,40,y5,b2,226.155003467,226.1552276611328,...,,,,,,,,,,
306,53.899383544921875 54.40927505493164 66.511924...,2461.784 3039.2222 2540.4285 8154.4355 2572.69...,5,PKKTE,FTMS,40,y5,b2,226.155003467,226.15432739257812,...,,,,,,,,,,
307,54.40829086303711 70.06476593017578 84.0803070...,3128.156 7825.7524 15576.507 2519.0374 2384.07...,4,PKKTE,FTMS,40,y5,b2;y3,226.155003467;377.203076167,226.15536499023438;377.2024841308594,...,,,,,,,,,,
308,54.40895080566406 59.92170333862305 70.0650405...,945.8575 1072.6782 2115.838 964.55164 2905.835...,5,PKKTE,FTMS,40,y5,b2,226.155003467,226.15496826171875,...,,,,,,,,,,


In [31]:
# concatenate all spectra (series) as well
all_series = [series1, series2, series3, series4, series5, series6, series7]
concat_series = pd.concat(all_series).reset_index(drop=True)
concat_series

0      <pyopenms.pyopenms_5.MSSpectrum object at 0x00...
1      <pyopenms.pyopenms_5.MSSpectrum object at 0x00...
2      <pyopenms.pyopenms_5.MSSpectrum object at 0x00...
3      <pyopenms.pyopenms_5.MSSpectrum object at 0x00...
4      <pyopenms.pyopenms_5.MSSpectrum object at 0x00...
                             ...                        
305    <pyopenms.pyopenms_5.MSSpectrum object at 0x00...
306    <pyopenms.pyopenms_5.MSSpectrum object at 0x00...
307    <pyopenms.pyopenms_5.MSSpectrum object at 0x00...
308    <pyopenms.pyopenms_5.MSSpectrum object at 0x00...
309    <pyopenms.pyopenms_5.MSSpectrum object at 0x00...
Length: 310, dtype: object

In [37]:
# to run csv function 'charge' needs to be replaced with 'precursor_charge' ?
concat_df.rename(columns = {'charge':'precursor_charge'}, inplace = True)

#df2 = concat_df[['modified_sequence', 'precursor_charge', 'collision_energy']].copy()

concat_df.assign(peptides_in=concat_df['modified_sequence'], precursor_charge_in=concat_df['precursor_charge'], collision_energy_in=concat_df['collision_energy'])
    
    
#concat_df['peptides_in'] = concat_df['modified_sequence']
#concat_df.rename(columns = {'charge':'precursor_charge_in'}, inplace = True)
#concat_df.rename(columns = {'collision_energy':'collision_energy_in'}, inplace = True)
# create dictionary of data
data = csv_training(concat_df, concat_series)
print(data)
# convert dictionary data to hdf5 
to_hdf5(data, 'hdf5_data.hdf5')

{'collision_energy_aligned_normed': array([[0.1 ],
       [0.1 ],
       [0.1 ],
       [0.1 ],
       [0.1 ],
       [0.1 ],
       [0.1 ],
       [0.1 ],
       [0.1 ],
       [0.1 ],
       [0.1 ],
       [0.1 ],
       [0.1 ],
       [0.1 ],
       [0.1 ],
       [0.1 ],
       [0.1 ],
       [0.1 ],
       [0.1 ],
       [0.1 ],
       [0.1 ],
       [0.1 ],
       [0.1 ],
       [0.1 ],
       [0.1 ],
       [0.1 ],
       [0.1 ],
       [0.1 ],
       [0.1 ],
       [0.1 ],
       [0.1 ],
       [0.1 ],
       [0.1 ],
       [0.1 ],
       [0.1 ],
       [0.1 ],
       [0.1 ],
       [0.1 ],
       [0.1 ],
       [0.1 ],
       [0.1 ],
       [0.15],
       [0.15],
       [0.15],
       [0.15],
       [0.15],
       [0.15],
       [0.15],
       [0.15],
       [0.15],
       [0.15],
       [0.15],
       [0.15],
       [0.15],
       [0.15],
       [0.15],
       [0.15],
       [0.15],
       [0.15],
       [0.15],
       [0.15],
       [0.15],
       [0.15],
       [0.15],
    

In [25]:
len(data['collision_energy_aligned_normed'])

228